In [1]:
!git clone https://github.com/guijiejie/DCMD-main.git

Cloning into 'DCMD-main'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 74 (delta 22), reused 48 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 262.60 KiB | 3.55 MiB/s, done.
Resolving deltas: 100% (22/22), done.


# **Pip**

In [2]:
!pip install appdirs==1.4.4 docker-pycreds==0.4.0 gitdb==4.0.10 gitpython==3.1.32 joblib==1.3.1 numpy==1.25.2 pathtools==0.1.2 protobuf==4.23.4 scikit-learn==1.3.0 scipy==1.11.1 sentry-sdk==1.29.2 setproctitle==1.3.2 smmap==5.0.0 threadpoolctl==3.2.0 wandb==0.15.8
!pip install --upgrade pytorch-lightning pyyaml wandb pandas numpy matplotlib matplotlib-inline scikit-learn tqdm
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 76.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 80.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 45.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.9 MB/s eta 0:00:00:00:01
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8793 sha256=492ea6045228194d82e1aea9f9d595

# **Replace code to run properly**

In [3]:
%%writefile /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml 
### Experiment configuration

## General settings
split: 'train' # data split; choices ['train', 'test']
debug: false # if true, load only a few data samples
seed: 999
validation: false # use validation; only for UBnormal
use_hr: false # for validation and test on UBnormal

## Computational resources
accelerator: 'gpu'
devices: [0] # indices of cuda devices to use

## Paths
dir_name: 'train_experiment' # name of the directory of the current experiment
data_dir: '/kaggle/input/avenue/Avenue' # path to the data
exp_dir: '/kaggle/working/DCMD-main/checkpoints' # path to the directory that will contain the current experiment directory
test_path: '/kaggle/input/avenue/Avenue/testing/test_frame_mask' # path to the test data
load_ckpt: 'best.ckpt' # name of the checkpoint to load at inference time
create_experiment_dir: true

## WANDB configuration
use_wandb: false
project_name: "project_name"
wandb_entity: "entity_name"
group_name: "group_name"
use_ema: false

##############################


### Model's configuration

## U-Net's configuration
dropout: 0. # probability of dropout
conditioning_strategy: 'inject'
## Rec configuration
h_dim: 512 # dimension of the bottleneck at the end of the encoder of the conditioning network
latent_dim: 256 # dimension of the latent space of the conditioning encoder
channels: [512,256,512] # channels for the encoder

##############################


### Training's configuration

## Diffusion's configuration
noise_steps: 10 # how many diffusion steps to perform

### Optimizer and scheduler's configuration
n_epochs: 10
opt_lr: 0.001

## Losses' configuration
loss_fn: 'smooth_l1' # loss function; choices ['mse', 'l1', 'smooth_l1']

##############################


### Inference's configuration
n_generated_samples: 50 # number of samples to generate
model_return_value: 'loss' # choices ['loss', 'poses', 'all']; if 'loss', the model will return the loss;
                           # if 'poses', the model will return the generated poses; 
                           # if 'all', the model will return both the loss and the generated poses
aggregation_strategy: 'best' # choices ['best', 'mean', 'median', 'random']; if 'best', the best sample will be selected; 
                             # if 'mean', the mean of loss of the samples will be selected; 
                             # if 'median', the median of the loss of the samples will be selected; 
                             # if 'random', a random sample will be selected;
                             # if 'mean_poses', the mean of the generated poses will be selected;
                             # if 'median_poses', the median of the generated poses will be selected;
                             # if 'all', all the generated poses will be selected
filter_kernel_size: 30 # size of the kernel to use for smoothing the anomaly score of each clip
frames_shift: 6 # it compensates the shift of the anomaly score due to the sliding window; 
                # in conjuction with pad_size and filter_kernel_size, it strongly depends on the dataset
save_tensors: false # if true, save the generated tensors for faster inference
load_tensors: false # if true, load the generated tensors for faster inference

##############################


### Dataset's configuration

## Important parameters
dataset_choice: 'HR-Avenue'
seg_len: 7 # length of the window (his+pre)
vid_res: [640,360]
batch_size: 2048
pad_size: 12 # size of the padding 

## Other parameters
headless: false # remove the keypoints of the head
hip_center: false # center the keypoints on the hip
kp18_format: false # use the 18 keypoints format
normalization_strategy: 'robust' # use 'none' to avoid normalization, 'robust' otherwise
num_coords: 2 # number of coordinates to use
num_transform: 5 # number of transformations to apply
num_workers: 4
seg_stride: 1
seg_th: 0
start_offset: 0
symm_range: true
use_fitted_scaler: false

## New configuration
n_his: 3
padding: 'LastFrame'
## translinear configuration
num_layers: 6
num_heads: 8
latent_dims: 512
loss_1_series_weight: 0.01
loss_1_prior_weight: 0
loss_2_series_weight: 0
loss_2_prior_weight: 0.01

Overwriting /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml


In [4]:
%%writefile /kaggle/working/DCMD-main/models/transformer.py
import torch
import torch.nn.functional as F
from torch import layer_norm, nn
import numpy as np
from typing import List, Tuple, Union

import math


def timestep_embedding(timesteps, dim, max_period=10000):
    """
    Create sinusoidal timestep embeddings.
    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
    half = dim // 2
    freqs = torch.exp(
        -math.log(max_period) * torch.arange(start=0, end=half, dtype=torch.float32) / half
    ).to(device=timesteps.device)
    args = timesteps[:, None].float() * freqs[None]
    embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    if dim % 2:
        embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
    return embedding


def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad for all the networks.

    Args:
        nets (nn.Module | list[nn.Module]): A list of networks or a single
            network.
        requires_grad (bool): Whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad


def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module


class StylizationBlock(nn.Module):

    def __init__(self, latent_dim, time_embed_dim, dropout):
        super().__init__()
        self.emb_layers = nn.Sequential(
            nn.SiLU(),
            nn.Linear(time_embed_dim, 2 * latent_dim),
        )
        self.norm = nn.LayerNorm(latent_dim)
        self.out_layers = nn.Sequential(
            nn.SiLU(),
            nn.Dropout(p=dropout),
            zero_module(nn.Linear(latent_dim, latent_dim)),
        )

    def forward(self, h, emb):
        """
        h: B, T, D
        emb: B, D
        """
        # B, 1, 2D
        emb_out = self.emb_layers(emb).unsqueeze(1)
        # scale: B, 1, D / shift: B, 1, D
        scale, shift = torch.chunk(emb_out, 2, dim=2)
        # B, T, D
        h = self.norm(h) * (1 + scale) + shift
        h = self.out_layers(h)
        return h


class FFN(nn.Module):

    def __init__(self, latent_dim, ffn_dim, dropout, time_embed_dim):
        super().__init__()
        self.linear1 = nn.Linear(latent_dim, ffn_dim)
        self.linear2 = zero_module(nn.Linear(ffn_dim, latent_dim))
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        self.proj_out = StylizationBlock(latent_dim, time_embed_dim, dropout)

    def forward(self, x, emb):
        """
            x: B, T, D (D=latent_dim)
        """
        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        y = x + self.proj_out(y, emb)
        return y


class TemporalSelfAttention(nn.Module):

    def __init__(self, n_frames, latent_dim, num_head, dropout, time_embed_dim, output_attention = True):
        super().__init__()
        self.num_head = num_head
        self.output_attention = output_attention
        self.norm = nn.LayerNorm(latent_dim)
        self.query = nn.Linear(latent_dim, latent_dim, bias=False)
        self.key = nn.Linear(latent_dim, latent_dim, bias=False)
        self.value = nn.Linear(latent_dim, latent_dim, bias=False)
        self.sigma_projection = nn.Linear(latent_dim, num_head, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.proj_out = StylizationBlock(latent_dim, time_embed_dim, dropout)
        n_frames = n_frames
        self.distances = torch.zeros((n_frames, n_frames)).cuda(0)

        for i in range(n_frames):
            for j in range(n_frames):
                self.distances[i][j] = abs(i - j)

    def forward(self, x, emb):
        """
        x: B, T, D (D=latent_dim)
        """
        B, T, D = x.shape
        H = self.num_head

        ## series-association
        # B, T, 1, D
        query = self.query(self.norm(x)).unsqueeze(2)
        # B, 1, T, D
        key = self.key(self.norm(x)).unsqueeze(1)
        # B, T, H, D/H
        query = query.view(B, T, H, -1)
        key = key.view(B, T, H, -1)
        scale = 1. / math.sqrt(D/H)
        # B, H, T, T
        scores = torch.einsum('bnhd,bmhd->bhnm', query, key) / math.sqrt(D // H)
        attention = scale * scores
        # B, H, T, T
        series = self.dropout(F.softmax(attention, dim=-1))

        ## prior-association
        sigma = self.sigma_projection(x).view(B, T, H)  # B, T, H
        sigma = sigma.transpose(1, 2)  # B T H ->  B H T
        sigma = torch.sigmoid(sigma * 5) + 1e-5
        sigma = torch.pow(3, sigma) - 1
        sigma = sigma.unsqueeze(-1).repeat(1, 1, 1, T)  # B, H, T, T
        prior = self.distances.unsqueeze(0).unsqueeze(0).repeat(sigma.shape[0], sigma.shape[1], 1, 1) # B, H, T, T
        prior = 1.0 / (math.sqrt(2 * math.pi) * sigma) * torch.exp(-prior ** 2 / 2 / (sigma ** 2)).cuda(0) # B, H, T, T

        # B, T, H, D/H
        value = self.value(self.norm(x)).view(B, T, H, -1)
        # B, T, D
        y = torch.einsum('bhnm,bmhd->bnhd', series, value).reshape(B, T, D)
        y = x + self.proj_out(y, emb)

        if self.output_attention:
            return y.contiguous(), series, prior, sigma
        else:
            return y.contiguous(), None

class TemporalDiffusionTransformerDecoderLayer(nn.Module):

    def __init__(self,
                 n_frames = 7,
                 latent_dim=16,
                 time_embed_dim=16,
                 ffn_dim=32,
                 num_head=4,
                 dropout=0.5
                 ):
        super().__init__()
        self.sa_block = TemporalSelfAttention(
            n_frames, latent_dim, num_head, dropout, time_embed_dim)
        self.ffn = FFN(latent_dim, ffn_dim, dropout, time_embed_dim)

    def forward(self, x, emb):
        x, series, prior, sigma = self.sa_block(x, emb)
        x = self.ffn(x, emb)
        return x, series, prior, sigma


class MotionTransformer(nn.Module):
    def __init__(self,
                 input_feats,
                 num_frames=7,
                 latent_dim=16,
                 ff_size=32,
                 num_layers=8,
                 num_heads=8,
                 dropout=0.2,
                 activation="gelu",
                 output_attention = True,
                 device: Union[str, torch.DeviceObjType] = 'cpu',
                 inject_condition: bool = False,
                 **kargs):
        super().__init__()


        self.input_feats = input_feats # 34
        self.num_frames = num_frames
        self.latent_dim = latent_dim
        self.ff_size = ff_size
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dropout = dropout
        self.activation = activation
        self.output_attention = output_attention
        self.device = device
        self.time_embed_dim = latent_dim
        self.inject_condition = inject_condition

        self.build_model()

    def build_model(self):
        self.sequence_embedding = nn.Parameter(torch.randn(self.num_frames, self.latent_dim))

        # Input Embedding
        self.joint_embed = nn.Linear(self.input_feats, self.latent_dim)
        self.cond_embed = nn.Linear(256, self.time_embed_dim)

        self.time_embed = nn.Sequential(
            nn.Linear(self.latent_dim, self.time_embed_dim),
            nn.SiLU(),
            nn.Linear(self.time_embed_dim, self.time_embed_dim),
        )

        self.temporal_decoder_blocks = nn.ModuleList()
        for i in range(self.num_layers):
            self.temporal_decoder_blocks.append(
                TemporalDiffusionTransformerDecoderLayer(
                    n_frames=self.num_frames,
                    latent_dim=self.latent_dim,
                    time_embed_dim=self.time_embed_dim,
                    ffn_dim=self.ff_size,
                    num_head=self.num_heads,
                    dropout=self.dropout,
                )
            )
        # Output Module
        self.out = zero_module(nn.Linear(self.latent_dim, self.input_feats))


    def forward(self, x, timesteps, condition_data:torch.Tensor=None):
        """
        x: B, T, D (D=C*V)
        """
        B, T = x.shape[0], x.shape[1]

        # B, latent_dim
        emb = self.time_embed(timestep_embedding(timesteps, self.latent_dim))

        # Add conditioning signal
        if self.inject_condition:
            condition_data = self.cond_embed(condition_data)
            emb = emb + condition_data

        # B, T, latent_dim
        h = self.joint_embed(x)
        h = h + self.sequence_embedding.unsqueeze(0)[:, :T, :]
        
        i = 0
        prelist = []
        series_list = []
        prior_list = []
        sigma_list = []
        for module in self.temporal_decoder_blocks:
            if i < (self.num_layers // 2):
                prelist.append(h)
                h, series, prior, sigmas = module(h, emb) # B, T, latent_dim
                series_list.append(series)
                prior_list.append(prior)
                sigma_list.append(sigmas)
            elif i >= (self.num_layers // 2):
                h, series, prior, sigmas = module(h, emb)
                h += prelist[-1]
                series_list.append(series)
                prior_list.append(prior)
                sigma_list.append(sigmas)
                prelist.pop()
            i += 1

        # B, T, C*V
        output = self.out(h).view(B, T, -1).contiguous()
        if self.output_attention:
            return output, series_list, prior_list, sigma_list
        return output

Overwriting /kaggle/working/DCMD-main/models/transformer.py


In [5]:
%%writefile /kaggle/working/DCMD-main/train_DCMD.py
import argparse
import os
import random

import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from models.dcmd import DCMD
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.strategies import DDPStrategy
from utils.argparser import init_args
from utils.dataset import get_dataset_and_loader
from utils.ema import EMACallback


if __name__== '__main__':

    # Parse command line arguments and load config file
    parser = argparse.ArgumentParser(description='Pose_AD_Experiment')
    parser.add_argument('-c', '--config', type=str, required=True,
                        default='/your_default_config_file_path')
    
    args = parser.parse_args()
    config_path = args.config
    args = yaml.load(open(args.config), Loader=yaml.FullLoader)
    args = argparse.Namespace(**args)
    args = init_args(args) 
    # Save config file to ckpt_dir
    os.system(f'cp {config_path} {os.path.join(args.ckpt_dir, "config.yaml")}')     
    
    # Set seeds    
    torch.manual_seed(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed) 
    pl.seed_everything(args.seed)

    # Set callbacks and logger
    if (hasattr(args, 'diffusion_on_latent') and args.stage == 'pretrain'):
        monitored_metric = 'pretrain_rec_loss'
        metric_mode = 'min'
    elif args.validation:
        monitored_metric = 'AUC'
        metric_mode = 'max'
    else:
        monitored_metric = 'loss'
        metric_mode = 'min'
    callbacks = [ModelCheckpoint(
                    dirpath=args.ckpt_dir,
                    save_top_k=2,
                    save_last=True,
                    monitor=monitored_metric,
                    mode=metric_mode
                )]

    
    callbacks += [EMACallback()] if args.use_ema else [] # Use to achieve exponential moving average
    
    if args.use_wandb:
        callbacks += [LearningRateMonitor(logging_interval='step')]
        wandb_logger = WandbLogger(project=args.project_name, group=args.group_name, entity=args.wandb_entity, 
                                   name=args.dir_name, config=vars(args), log_model='all')
    else:
        wandb_logger = False

    # Get dataset and loaders
    _, train_loader, _, val_loader = get_dataset_and_loader(args, split=args.split, validation=args.validation)
    
    # Initialize model and trainer
    model = DCMD(args)
    
    trainer = pl.Trainer(accelerator=args.accelerator, devices=args.devices, default_root_dir=args.ckpt_dir, max_epochs=args.n_epochs, 
                         logger=wandb_logger, callbacks=callbacks, strategy=DDPStrategy(find_unused_parameters=False),
                         log_every_n_steps=20, num_sanity_val_steps=0, deterministic=True)
    
    # Train the model    
    trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Overwriting /kaggle/working/DCMD-main/train_DCMD.py


In [6]:
%%writefile /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml
### Experiment configuration

## General settings
split: 'test' # data split; choices ['train', 'test']
debug: false # if true, load only a few data samples
seed: 999
validation: false # use validation; only for UBnormal
use_hr: false # for validation and test on UBnormal

## Computational resources
accelerator: 'gpu'
devices: [0] # indices of cuda devices to use

## Paths
dir_name: 'test_experiment' # name of the directory of the current experiment
data_dir: '/kaggle/input/avenue/Avenue' # path to the data
exp_dir: '/kaggle/working/DCMD-main/checkpoints' # path to the directory that will contain the current experiment directory
test_path: '/kaggle/input/avenue/Avenue/testing/test_frame_mask' # path to the test data
load_ckpt: 'last.ckpt' # name of the checkpoint to load at inference time
create_experiment_dir: false # if true, create a new directory for the current experiment

## WANDB configuration
use_wandb: false
project_name: "project_name"
wandb_entity: "entity_name"
group_name: "group_name"
use_ema: false

##############################


### Model's configuration

## U-Net's configuration
dropout: 0. # probability of dropout
conditioning_strategy: 'inject'
## Rec configuration
h_dim: 512 # dimension of the bottleneck at the end of the encoder of the conditioning network
latent_dim: 256 # dimension of the latent space of the conditioning encoder
channels: [512,256,512] # channels for the encoder

##############################


### Training's configuration

## Diffusion's configuration
noise_steps: 10 # how many diffusion steps to perform

### Optimizer and scheduler's configuration
n_epochs: 10
opt_lr: 0.001

## Losses' configuration
loss_fn: 'smooth_l1' # loss function; choices ['mse', 'l1', 'smooth_l1']

##############################


### Inference's configuration
n_generated_samples: 50 # number of samples to generate
model_return_value: 'loss' # choices ['loss', 'poses', 'all']; if 'loss', the model will return the loss;
                           # if 'poses', the model will return the generated poses; 
                           # if 'all', the model will return both the loss and the generated poses
aggregation_strategy: 'best' # choices ['best', 'mean', 'median', 'random']; if 'best', the best sample will be selected; 
                             # if 'mean', the mean of loss of the samples will be selected; 
                             # if 'median', the median of the loss of the samples will be selected; 
                             # if 'random', a random sample will be selected;
                             # if 'mean_poses', the mean of the generated poses will be selected;
                             # if 'median_poses', the median of the generated poses will be selected;
                             # if 'all', all the generated poses will be selected
filter_kernel_size: 30 # size of the kernel to use for smoothing the anomaly score of each clip
frames_shift: 6 # it compensates the shift of the anomaly score due to the sliding window; 
                # in conjuction with pad_size and filter_kernel_size, it strongly depends on the dataset
save_tensors: true # if true, save the generated tensors for faster inference
load_tensors: false # if true, load the generated tensors for faster inference

##############################


### Dataset's configuration

## Important parameters
dataset_choice: 'HR-Avenue'
seg_len: 7 # length of the window (cond+noised)
vid_res: [640,360]
batch_size: 2048
pad_size: 12 # size of the padding

## Other parameters
headless: false # remove the keypoints of the head
hip_center: false # center the keypoints on the hip
kp18_format: false # use the 18 keypoints format
normalization_strategy: 'robust' # use 'none' to avoid normalization, 'robust' otherwise
num_coords: 2 # number of coordinates to use
num_transform: 5 # number of transformations to apply
num_workers: 4
seg_stride: 1
seg_th: 0
start_offset: 0
symm_range: true
use_fitted_scaler: false

## New configuration
n_his: 3
padding: 'LastFrame'
## translinear configuration
num_layers: 6
num_heads: 8
latent_dims: 512
loss_1_series_weight: 0.01
loss_1_prior_weight: 0
loss_2_series_weight: 0
loss_2_prior_weight: 0.01

Overwriting /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml


In [7]:
%%writefile /kaggle/working/DCMD-main/models/dcmd.py
import argparse
import os
from math import prod
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt

from models.stsae.stsae import STSAE
from sklearn.metrics import roc_curve, roc_auc_score
from torch.optim import Adam
from tqdm import tqdm

from utils.diffusion_utils import Diffusion
from utils.eval_utils import (compute_var_matrix, filter_vectors_by_cond,
                              get_avenue_mask, get_hr_ubnormal_mask, pad_scores, score_process)
from utils.model_utils import processing_data, my_kl_loss
from models.transformer import MotionTransformer
from utils.tools import get_dct_matrix, generate_pad, padding_traj


class DCMD(pl.LightningModule):

    losses = {'l1': nn.L1Loss, 'smooth_l1': nn.SmoothL1Loss, 'mse': nn.MSELoss}
    conditioning_strategies = {'inject': 'inject'}

    def __init__(self, args: argparse.Namespace) -> None:
        """
        This class implements DCMD model.

        Args:
            args (argparse.Namespace): arguments containing the hyperparameters of the model
        """

        super(DCMD, self).__init__()

        ## Log the hyperparameters of the model
        self.save_hyperparameters(args)

        ## Set the internal variables of the model
        # Data parameters
        self.n_frames = args.seg_len
        self.num_coords = args.num_coords
        self.n_joints = self._infer_number_of_joint(args)

        ## Model parameters
        # Main network
        self.dropout = args.dropout
        self.conditioning_strategy = self.conditioning_strategies[args.conditioning_strategy]
        # Conditioning network
        self.cond_h_dim = args.h_dim
        self.cond_latent_dim = args.latent_dim
        self.cond_channels = args.channels
        self.cond_dropout = args.dropout

        ## Training and inference parameters
        self.learning_rate = args.opt_lr
        self.loss_fn = self.losses[args.loss_fn](reduction='none')
        self.noise_steps = args.noise_steps
        self.aggregation_strategy = args.aggregation_strategy
        self.n_generated_samples = args.n_generated_samples
        self.model_return_value = args.model_return_value
        self.gt_path = args.gt_path
        self.split = args.split
        self.use_hr = args.use_hr
        self.ckpt_dir = args.ckpt_dir
        self.save_tensors = args.save_tensors
        self.num_transforms = args.num_transform
        self.anomaly_score_pad_size = args.pad_size
        self.anomaly_score_filter_kernel_size = args.filter_kernel_size
        self.anomaly_score_frames_shift = args.frames_shift
        self.dataset_name = args.dataset_choice

        # New parameters
        self.n_his = args.n_his
        self.padding = args.padding
        self.num_layers = args.num_layers
        self.num_heads = args.num_heads
        self.latent_dims = args.latent_dims
        self.automatic_optimization = False
        self.loss_1_series_weight = args.loss_1_series_weight
        self.loss_1_prior_weight = args.loss_1_prior_weight
        self.loss_2_series_weight = args.loss_2_series_weight
        self.loss_2_prior_weight = args.loss_2_prior_weight
        self.idx_pad, self.zero_index = generate_pad(self.padding, self.n_his, self.n_frames-self.n_his)

        ## Set the noise scheduler for the diffusion process
        self._set_diffusion_variables()

        ## Build the model
        self.build_model()


    def build_model(self) -> None:
        """
        Build the model according to the specified hyperparameters.
        """

        # Prediction Model
        pre_model = MotionTransformer(
            input_feats=2 * self.n_joints,
            num_frames=self.n_frames,
            num_layers=self.num_layers,
            num_heads=self.num_heads,
            latent_dim=self.latent_dims,
            dropout=self.dropout,
            device=self.device,
            inject_condition=(self.conditioning_strategy == 'inject')
        )

        # Reconstruction Model
        rec_model = STSAE(
            c_in=self.num_coords,
            h_dim=self.cond_h_dim,
            latent_dim=self.cond_latent_dim,
            n_frames=self.n_his,
            dropout=self.cond_dropout,
            n_joints=self.n_joints,
            layer_channels=self.cond_channels,
            device=self.device)

        self.pre_model, self.rec_model = pre_model, rec_model


    def forward(self, input_data: List[torch.Tensor], aggr_strategy: str = None, return_: str = None) -> List[torch.Tensor]:
        """
        Forward pass of the model.
        """

        ## Unpack data: tensor_data is the input data, meta_out is a list of metadata
        tensor_data, meta_out = self._unpack_data(input_data)
        B = tensor_data.shape[0]

        ## Select frames to reconstruct and to predict
        history_data = tensor_data[:, :, :self.n_his, :]
        x_0 = padding_traj(history_data, self.padding, self.idx_pad, self.zero_index)

        generated_xs = []
        # Generate m future predictions
        for _ in range(self.n_generated_samples):

            ## Reconstruction —— AE model
            condition_embedding, rec_his_data = self.rec_model(history_data)

            ## Prediction —— diffusion model
            ## DCT transformation
            dct_m, idct_m = get_dct_matrix(self.n_frames)
            dct_m_all = dct_m.float().to(self.device)
            idct_m_all = idct_m.float().to(self.device)
            # (B, C, T, V) -> (B, T, V, C)
            x = x_0.permute(0, 2, 3, 1).contiguous()
            # (B, T, V, C) -> (B, T, C*V)
            x = x.reshape([x.shape[0], self.n_frames, -1])
            y = torch.matmul(dct_m_all, x)  # [B, T, C*V]

            ## Generate gaussian noise of the same shape as the y
            y_d = torch.randn_like(y, device=self.device)

            ## (t ∈ T, T-1, ..., 1)
            for i in reversed(range(1, self.noise_steps)):

                ### Prediction (Two branches)
                ## Set the time step
                t = torch.full(size=(B,), fill_value=i, dtype=torch.long, device=self.device)
                t_prev = torch.full(size=(B,), fill_value=i, dtype=torch.long, device=self.device)
                t_prev[0] = 0

                ## Generate gaussian noise of the same shape as the predicted noise
                noise_pre = torch.randn_like(y_d, device=self.device) if i > 1 else torch.zeros_like(y_d, device=self.device)

                ## First branch
                # Predict the noise
                predicted_noise_pre, series, prior, _ = self.pre_model(y_d, t, condition_data=condition_embedding)
                # Get the alpha and beta values and expand them to the shape of the predicted noise
                alpha_pre = self._alpha[t][:, None, None]
                alpha_hat_pre = self._alpha_hat[t][:, None, None]
                beta_pre = self._beta[t][:, None, None]
                # Recover the predicted sequence
                y_d = (1 / torch.sqrt(alpha_pre)) * (y_d - ((1 - alpha_pre) / (torch.sqrt(1 - alpha_hat_pre))) * predicted_noise_pre) \
                    + torch.sqrt(beta_pre) * noise_pre
                ## Second branch
                alpha_hat_prev = self._alpha_hat[t_prev][:, None, None]
                # Add noise
                y_n = (torch.sqrt(alpha_hat_prev) * y) + (torch.sqrt(1 - alpha_hat_prev) * noise_pre)
                ## Mask completion
                # Get M values
                mask = torch.zeros_like(x, device=self.device) # [batch, T, C*V]
                for m in range(0, self.n_his):
                    mask[:, m, :] = 1
                # iDCT transformation
                y_d_idct = torch.matmul(idct_m_all, y_d)
                y_n_idct = torch.matmul(idct_m_all, y_n)
                # mask-mul
                m_mul_y_n = torch.mul(mask, y_n_idct)
                m_mul_y_d = torch.mul((1-mask), y_d_idct)
                # together
                m_y = m_mul_y_d + m_mul_y_n
                # DCT again
                y_d = torch.matmul(dct_m_all, m_y)

            # iDCT
            pre_future_data = torch.matmul(idct_m_all, y_d)
            # (B, T, C*V) -> (B, T, V, C)
            pre_future_data = pre_future_data.reshape(pre_future_data.shape[0], pre_future_data.shape[1], -1, 2)
            # (B, T, V, C) -> (B, C, T, V)
            pre_future_data = pre_future_data.permute(0, 3, 1, 2).contiguous()
            # select future sequences
            pre_future_data = pre_future_data[:,:,self.n_his:,:]

            ## Reconstruction + Prediction
            xs = torch.cat((rec_his_data, pre_future_data), dim=2)

            generated_xs.append(xs)

        selected_x, loss_of_selected_x = self._aggregation_strategy(generated_xs, tensor_data, aggr_strategy)

        return self._pack_out_data(selected_x, loss_of_selected_x, [tensor_data] + meta_out, return_=return_)


    def training_step(self, batch: List[torch.Tensor], batch_idx: int) -> torch.float32:
        """
        Training step of the model.
        """

        ## Get the optimizer returned in configuration_optimizers()
        opt = self.optimizers()

        ## Unpack data: tensor_data is the input data
        tensor_data, _ = self._unpack_data(batch)

        ## Select frames to reconstruct and to predict
        history_data = tensor_data[:, :, :self.n_his, :] # Used for rec (first n_his)
        x_0 = tensor_data # Used for pre（all）

        ## Reconstruction
        # Encode the history data
        condition_embedding, rec_his_data = self.rec_model(history_data)
        # Compute the rec_loss
        rec_loss = torch.mean(self.loss_fn(rec_his_data, history_data))
        self.log('rec_loss', rec_loss)

        ## Prediction
        # DCT transformation
        dct_m, _ = get_dct_matrix(self.n_frames)
        dct_m_all = dct_m.float().to(self.device)
        # (B, C, T, V) -> (B, T, V, C)
        x = x_0.permute(0, 2, 3, 1).contiguous()
        # (B, T, V, C) -> (B, T, C*V)
        x = x.reshape([x.shape[0], self.n_frames, -1]) # [batch, T, C*V]
        y_0 = torch.matmul(dct_m_all, x)

        # Sample the time steps and corrupt the data
        t = self.noise_scheduler.sample_timesteps(y_0.shape[0]).to(self.device)
        y_t, pre_noise = self.noise_scheduler.noise_motion(y_0, t) # (B, T, C*(V-1))

        # Predict the noise
        pre_predicted_noise, series, prior, _ = self.pre_model(y_t, t, condition_data=condition_embedding)

        # Compute the pre_loss
        # Calculate Association discrepancy
        series_loss = 0.0
        prior_loss = 0.0
        for u in range(len(prior)):
            # Pdetach, S <-> Maximize
            series_loss += \
                (torch.mean(my_kl_loss(
                    series[u],
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)).detach()))
                + torch.mean(my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)).detach(),
                    series[u])))
            # P, Sdetach <-> Minimize
            prior_loss += \
                (torch.mean(my_kl_loss(
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)),
                    series[u].detach()))
                + torch.mean(my_kl_loss(
                    series[u].detach(),
                    (prior[u] / torch.unsqueeze(torch.sum(prior[u], dim=-1), dim=-1).repeat(1, 1, 1, self.n_frames)))))
        series_loss = series_loss / len(prior)
        prior_loss = prior_loss / len(prior)

        pre_loss = torch.mean(self.loss_fn(pre_predicted_noise, pre_noise))
        self.log('pre_loss', pre_loss)

        ## Compute loss1 & loss2
        loss1 = rec_loss + pre_loss \
                - self.loss_1_series_weight * series_loss \
                + self.loss_1_prior_weight * prior_loss
        self.log('loss1', loss1)
        loss2 = rec_loss + pre_loss \
                + self.loss_2_prior_weight * prior_loss \
                + self.loss_2_series_weight * series_loss
        self.log('loss2', loss2)

        ## Minimax strategy
        self.manual_backward(loss1, retain_graph=True)
        self.manual_backward(loss2)
        opt.step()
        opt.zero_grad()


    def test_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        """
        Test step of the model. It saves the output of the model and the input data as
        List[torch.Tensor]: [predicted poses and the loss, tensor_data, transformation_idx, metadata, actual_frames]

        Args:
            batch (List[torch.Tensor]): list containing the following tensors:
                                         - tensor_data: tensor of shape (B, C, T, V) containing the input sequences
                                         - transformation_idx
                                         - metadata
                                         - actual_frames
            batch_idx (int): index of the batch
        """

        self._test_output_list.append(self.forward(batch))
        return


    def on_test_epoch_start(self) -> None:
        """
        Called when the test epoch begins.
        """

        super().on_test_epoch_start()
        self._test_output_list = []
        return


    def on_test_epoch_end(self) -> float:
        """
        Test epoch end of the model.

        Returns:
            float: test auc score
        """

        out, gt_data, trans, meta, frames = processing_data(self._test_output_list)
        del self._test_output_list
        if self.save_tensors:
            tensors = {'prediction': out, 'gt_data': gt_data,
                       'trans': trans, 'metadata': meta, 'frames': frames}
            self._save_tensors(tensors, split_name=self.split, aggr_strategy=self.aggregation_strategy, n_gen=self.n_generated_samples)
        auc_score = self.post_processing(out, gt_data, trans, meta, frames)
        self.log('AUC', auc_score)
        return auc_score


    def validation_step(self, batch: List[torch.Tensor], batch_idx: int) -> None:
        """
        Validation step of the model. It saves the output of the model and the input data as
        List[torch.Tensor]: [predicted poses and the loss, tensor_data, transformation_idx, metadata, actual_frames]

        Args:
            batch (List[torch.Tensor]): list containing the following tensors:
                                         - tensor_data: tensor of shape (B, C, T, V) containing the input sequences
                                         - transformation_idx
                                         - metadata
                                         - actual_frames
            batch_idx (int): index of the batch
        """

        self._validation_output_list.append(self.forward(batch))
        return


    def on_validation_epoch_start(self) -> None:
        """
        Called when the test epoch begins.
        """

        super().on_validation_epoch_start()
        self._validation_output_list = []
        return


    def on_validation_epoch_end(self) -> float:
        """
        Validation epoch end of the model.

        Returns:
            float: validation auc score
        """

        out, gt_data, trans, meta, frames = processing_data(self._validation_output_list)
        del self._validation_output_list
        if self.save_tensors:
            tensors = {'prediction': out, 'gt_data': gt_data,
                       'trans': trans, 'metadata': meta, 'frames': frames}
            self._save_tensors(tensors, split_name=self.split, aggr_strategy=self.aggregation_strategy, n_gen=self.n_generated_samples)
        auc_score = self.post_processing(out, gt_data, trans, meta, frames)
        self.log('AUC', auc_score, sync_dist=True)
        return auc_score


    def configure_optimizers(self) -> Dict:
        """
        Configure the optimizers and the learning rate schedulers.

        Returns:
            Dict: dictionary containing the optimizers, the learning rate schedulers and the metric to monitor
        """

        optimizer = Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99, last_epoch=-1)

        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'AUC'}


    def post_processing(self, out: np.ndarray, gt_data: np.ndarray, trans: np.ndarray, meta: np.ndarray, frames: np.ndarray) -> float:
        """
        Post processing of the model.

        Args:
            out (np.ndarray): output of the model
            gt_data (np.ndarray): ground truth data
            trans (np.ndarray): transformation index
            meta (np.ndarray): metadata
            frames (np.ndarray): frame indexes of the data

        Returns:
            float: auc score
        """

        all_gts = [file_name for file_name in os.listdir(self.gt_path) if file_name.endswith('.npy')]
        all_gts = sorted(all_gts)
        scene_clips = [(int(fn.split('_')[0]), int(fn.split('_')[1].split('.')[0])) for fn in all_gts]
        hr_ubnormal_masked_clips = get_hr_ubnormal_mask(self.split) if (self.use_hr and (self.dataset_name == 'UBnormal')) else {}
        hr_avenue_masked_clips = get_avenue_mask() if self.dataset_name == 'HR-Avenue' else {}

        num_transform = self.num_transforms
        model_scores_transf = {}
        dataset_gt_transf = {}

        for transformation in tqdm(range(num_transform)):
            # iterating over each transformation T

            dataset_gt = []
            model_scores = []
            cond_transform = (trans == transformation)

            out_transform, gt_data_transform, meta_transform, frames_transform = filter_vectors_by_cond([out, gt_data, meta, frames], cond_transform)

            for idx in range(len(all_gts)):
                # iterating over each clip C with transformation T

                scene_idx, clip_idx = scene_clips[idx]

                gt = np.load(os.path.join(self.gt_path, all_gts[idx]))
                n_frames = gt.shape[0]

                cond_scene_clip = (meta_transform[:, 0] == scene_idx) & (meta_transform[:, 1] == clip_idx)
                out_scene_clip, gt_scene_clip, meta_scene_clip, frames_scene_clip = filter_vectors_by_cond([out_transform, gt_data_transform,
                                                                                                            meta_transform, frames_transform],
                                                                                                           cond_scene_clip)

                figs_ids = sorted(list(set(meta_scene_clip[:, 2])))
                error_per_person = []
                error_per_person_max_loss = []

                for fig in figs_ids:
                    # iterating over each actor A in each clip C with transformation T

                    cond_fig = (meta_scene_clip[:, 2] == fig)
                    out_fig, _, frames_fig = filter_vectors_by_cond([out_scene_clip, gt_scene_clip, frames_scene_clip], cond_fig)
                    loss_matrix = compute_var_matrix(out_fig, frames_fig, n_frames)
                    fig_reconstruction_loss = np.nanmax(loss_matrix, axis=0)
                    if self.anomaly_score_pad_size != -1:
                        fig_reconstruction_loss = pad_scores(fig_reconstruction_loss, gt, self.anomaly_score_pad_size)

                    error_per_person.append(fig_reconstruction_loss)
                    error_per_person_max_loss.append(max(fig_reconstruction_loss))

                clip_score = np.stack(error_per_person, axis=0)
                clip_score_log = np.log1p(clip_score)
                clip_score = np.mean(clip_score, axis=0) + (np.amax(clip_score_log, axis=0)-np.amin(clip_score_log, axis=0))

                # removing the non-HR frames for UBnormal dataset
                if (scene_idx, clip_idx) in hr_ubnormal_masked_clips:
                    clip_score = clip_score[hr_ubnormal_masked_clips[(scene_idx, clip_idx)]]
                    gt = gt[hr_ubnormal_masked_clips[(scene_idx, clip_idx)]]

                # removing the non-HR frames for Avenue dataset
                if clip_idx in hr_avenue_masked_clips:
                    clip_score = clip_score[np.array(hr_avenue_masked_clips[clip_idx])==1]
                    gt = gt[np.array(hr_avenue_masked_clips[clip_idx])==1]

                # Abnormal score per frame
                clip_score = score_process(clip_score, self.anomaly_score_frames_shift, self.anomaly_score_filter_kernel_size)
                model_scores.append(clip_score)

                dataset_gt.append(gt)

            model_scores = np.concatenate(model_scores, axis=0)

            dataset_gt = np.concatenate(dataset_gt, axis=0)

            model_scores_transf[transformation] = model_scores
            dataset_gt_transf[transformation] = dataset_gt

        # aggregating the anomaly scores for all transformations
        pds = np.mean(np.stack(list(model_scores_transf.values()), 0), 0)
        gt = dataset_gt_transf[0]

        # computing the AUC
        auc = roc_auc_score(gt, pds)

        return auc


    def test_on_saved_tensors(self, split_name: str) -> float:
        """
        Skip the prediction step and test the model on the saved tensors.

        Args:
            split_name (str): split name (val, test)

        Returns:
            float: auc score
        """

        tensors = self._load_tensors(split_name, self.aggregation_strategy, self.n_generated_samples)
        auc_score = self.post_processing(tensors['prediction'], tensors['gt_data'], tensors['trans'],
                                         tensors['metadata'], tensors['frames'])
        print(f'AUC score: {auc_score:.6f}')
        return auc_score


    ## Helper functions

    def _aggregation_strategy(self, generated_xs: List[torch.Tensor], input_sequence: torch.Tensor, aggr_strategy: str) -> Tuple[torch.Tensor]:
        """
        Aggregates the generated samples and returns the selected one and its reconstruction error.
        Strategies:
            - all: returns all the generated samples
            - random: returns a random sample
            - best: returns the sample with the lowest reconstruction loss
            - worst: returns the sample with the highest reconstruction loss
            - mean: returns the mean of the losses of the generated samples
            - median: returns the median of the losses of the generated samples
            - mean_pose: returns the mean of the generated samples
            - median_pose: returns the median of the generated samples

        Args:
            generated_xs (List[torch.Tensor]): list of generated samples
            input_sequence (torch.Tensor): ground truth sequence
            aggr_strategy (str): aggregation strategy

        Raises:
            ValueError: if the aggregation strategy is not valid

        Returns:
            Tuple[torch.Tensor]: selected sample and its reconstruction error
        """

        aggr_strategy = self.aggregation_strategy if aggr_strategy is None else aggr_strategy
        if aggr_strategy == 'random':
            return generated_xs[np.random.randint(len(generated_xs))], None # Added None as it was missing

        B, repr_shape = input_sequence.shape[0], input_sequence.shape[1:]
        compute_loss = lambda x: torch.mean(self.loss_fn(x, input_sequence).reshape(-1, prod(repr_shape)), dim=-1)
        losses = [compute_loss(x) for x in generated_xs]

        if aggr_strategy == 'all':
            dims_idxs = list(range(2, len(repr_shape)+2))
            dims_idxs = [1, 0] + dims_idxs
            selected_x = torch.stack(generated_xs).permute(*dims_idxs)
            loss_of_selected_x = torch.stack(losses).permute(1, 0)
        elif aggr_strategy == 'mean':
            selected_x = None
            loss_of_selected_x = torch.mean(torch.stack(losses), dim=0)
        elif aggr_strategy == 'mean_pose':
            selected_x = torch.mean(torch.stack(generated_xs), dim=0)
            loss_of_selected_x = compute_loss(selected_x)
        elif aggr_strategy == 'median':
            loss_of_selected_x, _ = torch.median(torch.stack(losses), dim=0)
            selected_x = None
        elif aggr_strategy == 'median_pose':
            selected_x, _ = torch.median(torch.stack(generated_xs), dim=0)
            loss_of_selected_x = compute_loss(selected_x)
        elif aggr_strategy == 'best' or aggr_strategy == 'worst':
            strategy = (lambda x, y: x < y) if aggr_strategy == 'best' else (lambda x, y: x > y)
            loss_of_selected_x = torch.full((B,), fill_value=(1e10 if aggr_strategy == 'best' else -1.), device=self.device)
            selected_x = torch.zeros((B, *repr_shape)).to(self.device)

            for i in range(len(generated_xs)):
                mask = strategy(losses[i], loss_of_selected_x)
                loss_of_selected_x[mask] = losses[i][mask]
                selected_x[mask] = generated_xs[i][mask]
        elif 'quantile' in aggr_strategy:
            q = float(aggr_strategy.split(':')[-1])
            loss_of_selected_x = torch.quantile(torch.stack(losses), q, dim=0)
            selected_x = None
        else:
            raise ValueError(f'Unknown aggregation strategy {aggr_strategy}')

        # Ensuring selected_x and loss_of_selected_x are always returned
        if selected_x is None and loss_of_selected_x is None:
             # Default to mean loss if strategy doesn't return both
             loss_of_selected_x = torch.mean(torch.stack(losses), dim=0)


        return selected_x, loss_of_selected_x


    def _infer_number_of_joint(self, args: argparse.Namespace) -> int:
        """
        Infer the number of joints based on the dataset parameters.

        Args:
            args (argparse.Namespace): arguments containing the hyperparameters of the model

        Returns:
            int: number of joints
        """

        if args.headless:
            joints_to_consider = 14
        elif args.kp18_format:
            joints_to_consider = 18
        else:
            joints_to_consider = 17
        return joints_to_consider


    def _load_tensors(self, split_name: str, aggr_strategy: str, n_gen: int) -> Dict[str, torch.Tensor]:
        """
        Loads the tensors from the experiment directory.

        Args:
            split_name (str): name of the split (train, val, test)
            aggr_strategy (str): aggregation strategy
            n_gen (int): number of generated samples

        Returns:
            Dict[str, torch.Tensor]: dictionary containing the tensors. The keys are inferred from the file names.
        """

        name = 'saved_tensors_{}_{}_{}'.format(split_name, aggr_strategy, n_gen)
        path = os.path.join(self.ckpt_dir, name)
        if not os.path.exists(path):
            os.mkdir(path)
        tensor_files = os.listdir(path)
        tensors = {}
        for t_file in tensor_files:
            t_name = t_file.split('.')[0]
            tensors[t_name] = torch.load(os.path.join(path, t_file))
        return tensors


    def _pack_out_data(self, selected_x: torch.Tensor, loss_of_selected_x: torch.Tensor, additional_out: List[torch.Tensor], return_: str) -> List[torch.Tensor]:
        """
        Packs the output data according to the return_ argument.

        Args:
            selected_x (torch.Tensor): generated samples selected among the others according to the aggregation strategy
            loss_of_selected_x (torch.Tensor): loss of the selected samples
            additional_out (List[torch.Tensor]): additional output data (ground truth, meta-data, etc.)
            return_ (str): return strategy. Can be 'pose', 'loss', 'all'

        Raises:
            ValueError: if return_ is None and self.model_return_value is None

        Returns:
            List[torch.Tensor]: output data
        """

        if return_ is None:
            if self.model_return_value is None:
                raise ValueError('Either return_ or self.model_return_value must be set')
            else:
                return_ = self.model_return_value

        if return_ == 'poses':
            out = [selected_x]
        elif return_ == 'loss':
            out = [loss_of_selected_x]
        elif return_ == 'all':
            # Check if both are available before adding to the list
            out = []
            if loss_of_selected_x is not None:
                out.append(loss_of_selected_x)
            if selected_x is not None:
                 out.append(selected_x)

        return out + additional_out


    def _save_tensors(self, tensors: Dict[str, torch.Tensor], split_name: str, aggr_strategy: str, n_gen: int) -> None:
        """
        Saves the tensors in the experiment directory.

        Args:
            tensors (Dict[str, torch.Tensor]): tensors to save
            split_name (str): name of the split (val, test)
            aggr_strategy (str): aggregation strategy
            n_gen (int): number of generated samples
        """

        name = 'saved_tensors_{}_{}_{}'.format(split_name, aggr_strategy, n_gen)
        path = os.path.join(self.ckpt_dir, name)
        if not os.path.exists(path):
            os.mkdir(path)
        for t_name, tensor in tensors.items():
            torch.save(tensor, os.path.join(path, t_name + '.pt'))


    def _set_diffusion_variables(self) -> None:
        """
        Sets the diffusion variables.
        """

        self.noise_scheduler = Diffusion(noise_steps=self.noise_steps, n_joints=self.n_joints,
                                         device=self.device, time=self.n_frames)
        self._beta_ = self.noise_scheduler.schedule_noise()
        self._alpha_ = (1. - self._beta_)
        self._alpha_hat_ = torch.cumprod(self._alpha_, dim=0)

    def _unpack_data(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        """
        Unpacks the data.

        Args:
            x (torch.Tensor): list containing the input data, the transformation index, the metadata and the actual frames.

        Returns:
            Tuple[torch.Tensor, List[torch.Tensor]]: input data, list containing the transformation index, the metadata and the actual frames.
        """
        tensor_data = x[0].to(self.device)
        transformation_idx = x[1]
        metadata = x[2]
        actual_frames = x[3]
        meta_out = [transformation_idx, metadata, actual_frames]
        return tensor_data, meta_out


    @property
    def _beta(self) -> torch.Tensor:
        return self._beta_.to(self.device)


    @property
    def _alpha(self) -> torch.Tensor:
        return self._alpha_.to(self.device)


    @property
    def _alpha_hat(self) -> torch.Tensor:
        return self._alpha_hat_.to(self.device)

Overwriting /kaggle/working/DCMD-main/models/dcmd.py


In [8]:
%%writefile /kaggle/working/DCMD-main/eval_DCMD.py
import argparse
import os

import pytorch_lightning as pl
import yaml
from models.dcmd import DCMD
from utils.argparser import init_args
from utils.dataset import get_dataset_and_loader



if __name__== '__main__':
    
    # Parse command line arguments and load config file
    parser = argparse.ArgumentParser(description='DCMD')
    parser.add_argument('-c', '--config', type=str, required=True)
    args = parser.parse_args()
    args = yaml.load(open(args.config), Loader=yaml.FullLoader)
    args = argparse.Namespace(**args)
    args = init_args(args)

    # Initialize the model
    model = DCMD(args)
    
    if args.load_tensors:
        # Load tensors and test
        model.test_on_saved_tensors(split_name=args.split)
    else:
        # Load test data
        print('Loading data and creating loaders.....')
        ckpt_path = '/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/last.ckpt'
        dataset, loader, _, _ = get_dataset_and_loader(args, split=args.split)
        
        # Initialize trainer and test
        trainer = pl.Trainer(accelerator=args.accelerator, devices=args.devices[:1],
                             default_root_dir=args.ckpt_dir, max_epochs=1, logger=False)
        out = trainer.test(model, dataloaders=loader, ckpt_path=ckpt_path)

Overwriting /kaggle/working/DCMD-main/eval_DCMD.py


In [9]:
%%writefile /kaggle/working/DCMD-main/utils/get_robust_data.py
import os
import numpy as np
import pickle

from copy import deepcopy

import torch

from utils.data import load_trajectories, extract_global_features
from utils.data import change_coordinate_system, scale_trajectories, aggregate_autoencoder_data
from utils.data import input_trajectories_missing_steps
from utils.preprocessing import remove_short_trajectories, aggregate_rnn_autoencoder_data


def save_scaler(scaler, path):
    with open(path, 'wb') as scaler_file:
        pickle.dump(scaler, scaler_file)
    
        
def load_scaler(path):
    with open(path, 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    return scaler



# Load trajectory data and convert it into a format suitable for RNN autoencoders for training and testing of joint models
def data_of_combined_model(**args):
    # General
    exp_dir = args.get('exp_dir', '')
    split = args.get('split', 'train')
    normalize_pose = args.get('normalize_pose', False)
    trajectories_path = args.get('trajectories_path', '')
    include_global = args.get('include_global', True)
    debug = args.get('debug', False)
    if 'train' in split:
      # đổi thành path khác trong folder data nếu train dataset khác
        subfolder = '/kaggle/input/avenue/Avenue/training'
    elif 'test' in split:
        subfolder = '/kaggle/input/avenue/Avenue/testing'
    else:
        subfolder = 'validating'
    trajectories_path = os.path.join(trajectories_path, f'{subfolder}/trajectories')
    video_resolution = args.get('vid_res', (1080,720))
    video_resolution = np.array(video_resolution, dtype=np.float32)
    # Architecture
    reconstruct_original_data = args.get('reconstruct_original_data', False) 
    input_length = args.get('seg_len', 12)
    seg_stride = args.get('seg_stride', 1) - 1 
    pred_length = 0 
    # Training
    input_missing_steps = False # args.input_missing_steps
    
    if normalize_pose == True:
        global_normalisation_strategy = args.get('normalization_strategy', 'robust')
        local_normalisation_strategy = args.get('normalization_strategy', 'robust')
        out_normalisation_strategy = args.get('normalization_strategy', 'robust')


    trajectories = load_trajectories(trajectories_path, debug=debug, split=split)
    print('\nLoaded %d trajectories.' % len(trajectories))

    trajectories = remove_short_trajectories(trajectories, input_length=input_length,
                                             input_gap=seg_stride, pred_length=pred_length)
    print('\nRemoved short trajectories. Number of trajectories left: %d.' % len(trajectories))

    # trajectories, trajectories_val = split_into_train_and_test(trajectories, train_ratio=0.8, seed=42)

    if input_missing_steps:
        trajectories = input_trajectories_missing_steps(trajectories)
        print('\nInputted missing steps of trajectories.')

    # Global
    if include_global:
        global_trajectories = extract_global_features(deepcopy(trajectories), video_resolution=video_resolution)

        global_trajectories = change_coordinate_system(global_trajectories, video_resolution=video_resolution,
                                                        coordinate_system='global', invert=False)

        print('\nChanged global trajectories\'s coordinate system to global.')
        
        X_global, y_global, X_global_meta, y_global_meta = aggregate_rnn_autoencoder_data(global_trajectories, 
                                                                                        input_length=input_length,
                                                                                        input_gap=seg_stride, pred_length=pred_length, 
                                                                                        return_ids=True)
        
        if normalize_pose == True:
            # nếu test avenue thì dùng dòng này
            #/content/checkpoints/HR-Avenue/train_experiment/local_robust.pickle
            # default: scaler_path = os.path.join(exp_dir, f'global_{global_normalisation_strategy}.pickle')
            scaler_path = '/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/local_robust.pickle'
            if split == 'train':
                _, global_scaler = scale_trajectories(aggregate_autoencoder_data(global_trajectories),
                                                    strategy=global_normalisation_strategy)
                save_scaler(global_scaler, scaler_path)
            else:
                global_scaler = load_scaler(scaler_path)

            X_global, _ = scale_trajectories(X_global, scaler=global_scaler, strategy=global_normalisation_strategy)
            
            if y_global is not None:
                y_global, _ = scale_trajectories(y_global, scaler=global_scaler,
                                                strategy=global_normalisation_strategy)
                
            print('\nNormalised global trajectories using the %s normalisation strategy.' % global_normalisation_strategy)
    
    else:
        X_global, X_global_meta = None, None
    
    # Local
    local_trajectories = deepcopy(trajectories) if reconstruct_original_data else trajectories

    local_trajectories = change_coordinate_system(local_trajectories, video_resolution=video_resolution,
                                                  coordinate_system='bounding_box_centre', invert=False)

    print('\nChanged local trajectories\'s coordinate system to bounding_box_centre.')

    X_local, y_local, X_local_meta, y_local_meta = aggregate_rnn_autoencoder_data(local_trajectories, input_length=input_length, 
                                                                                  input_gap=seg_stride, pred_length=pred_length,
                                                                                  return_ids=True)
    
    if normalize_pose == True:
        #scaler_path = '/content/drive/MyDrive/DCMD-main-main/checkpoints/Avenue/test_experiment/local_robust.pickle'
        scaler_path = os.path.join(exp_dir, f'local_{local_normalisation_strategy}.pickle')

        if split == 'train':
            _, local_scaler = scale_trajectories(aggregate_autoencoder_data(local_trajectories),
                                                strategy=local_normalisation_strategy)
            save_scaler(local_scaler, scaler_path)
        else:
            local_scaler = load_scaler("/kaggle/input/checkpoints/kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment/local_robust.pickle")

        X_local, _ = scale_trajectories(X_local, scaler=local_scaler, strategy=local_normalisation_strategy)

        if y_local is not None:
            y_local, _ = scale_trajectories(y_local, scaler=local_scaler, strategy=local_normalisation_strategy)
        
        print('\nNormalised local trajectories using the %s normalisation strategy.' % local_normalisation_strategy)

    # (Optional) Reconstruct the original data
    if reconstruct_original_data:
        print('\nReconstruction/Prediction target is the original data.')
        out_trajectories = trajectories
        
        out_trajectories = change_coordinate_system(out_trajectories, video_resolution=video_resolution,
                                                    coordinate_system='global', invert=False)
    
        print('\nChanged target trajectories\'s coordinate system to global.')
        
        scaler_path = os.path.join(exp_dir, f'out_{out_normalisation_strategy}.pickle')
    
        if split == 'train':
            _, out_scaler = scale_trajectories(aggregate_autoencoder_data(out_trajectories),
                                               strategy=out_normalisation_strategy)
            save_scaler(out_scaler, scaler_path)
        else:
            out_scaler = load_scaler(scaler_path)
        
        ######## X_out_{}, y_out_{} numpy arrays

        X_out, y_out, X_out_meta, y_out_meta = aggregate_rnn_autoencoder_data(out_trajectories, input_length=input_length, 
                                                                              input_gap=seg_stride, pred_length=pred_length,
                                                                              return_ids=True)

        X_out, _ = scale_trajectories(X_out, scaler=out_scaler, strategy=out_normalisation_strategy)
        
        if y_out is not None:
            y_out, _ = scale_trajectories(y_out, scaler=out_scaler, strategy=out_normalisation_strategy)
            
        print('\nNormalised target trajectories using the %s normalisation strategy.' % out_normalisation_strategy)
        
            
    if pred_length > 0:
        
        if reconstruct_original_data:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (X_out, X_out_meta), \
                   (y_global, y_global_meta), \
                   (y_local, y_local_meta), \
                   (y_out, y_out_meta) 
        else:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (y_global, y_global_meta), \
                   (y_local, y_local_meta)
    else:
        if reconstruct_original_data:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta), \
                   (X_out, X_out_meta)
        else:
            return (X_global, X_global_meta), \
                   (X_local, X_local_meta)

Overwriting /kaggle/working/DCMD-main/utils/get_robust_data.py


# **Train** 

In [25]:
!python /kaggle/working/DCMD-main/train_DCMD.py --config /kaggle/working/DCMD-main/config/Avenue/dcmd_train.yaml

Experiment directories created in /kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment

Loaded 2786 trajectories.

Removed short trajectories. Number of trajectories left: 1735.

Changed local trajectories's coordinate system to bounding_box_centre.

Normalised local trajectories using the robust normalisation strategy.
STSE: Adjacency matrix not provided. Initializing Graph with layout='h36m'.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/DCMD-main/checkpoints/HR-Avenue/train_experiment exists and is not empty.
Epoch 0:   0%|                                          | 0/185 [00:00<?, ?it/s]Warning: Encoder output T,V (2,17) differs from expected (1,17). Check strides and n_joints.
[rank0]: Traceback (most recent cal

If download checkpoint needed, run this code and download checkpoints.zip file in Output

In [ ]:
# 1. Zip toàn bộ folder checkpoints
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/DCMD-main/checkpoints

# 2. (tuỳ chọn) kiểm tra xem zip đã tạo xong chưa
!ls -lh /kaggle/working/checkpoints.zip


# **Testing**

In [ ]:
import os
os.makedirs('/kaggle/working/DCMD-main/checkpoints/HR-Avenue/test_experiment', exist_ok=True)

In [ ]:
!python /kaggle/working/DCMD-main/eval_DCMD.py --config /kaggle/working/DCMD-main/config/Avenue/dcmd_test.yaml

In [ ]:
# 1. Zip toàn bộ folder checkpoints
!zip -r /kaggle/working/checkpoints.zip /kaggle/working/DCMD-main/checkpoints



> Đổi phương pháp cho STGCN

In [10]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:0000:01


In [24]:
%%writefile /kaggle/working/DCMD-main/models/stsae/stsae.py
import torch.nn as nn
import torch
import numpy as np # Cần thiết nếu adjacency_matrix là numpy array ban đầu
from typing import List, Tuple, Union

# Đảm bảo đường dẫn import này chính xác dựa trên cấu trúc thư mục của bạn
from models.common.components import Encoder, Decoder 
# Graph sẽ được import bên trong STSE nếu adjacency_matrix is None

class STSE(nn.Module):
    def __init__(
        self,
        c_in: int,
        h_dim: int = 32, # hidden_dimension cho output của Encoder GAT-TCN stack
        latent_dim: int = 64, # Kích thước của bottleneck latent space Z
        n_frames: int = 12,
        n_joints: int = 17,
        layer_channels: List[int] = [64, 128, 256], # Channels cho các block trung gian
        dropout: float = 0.3,
        device: Union[str, torch.device] = 'cpu', # Sửa DeviceObjType thành torch.device
        adjacency_matrix=None,
        num_heads_gat: int = 4,
        encoder_temporal_strides: List[int] = None, # Thêm cấu hình strides
        decoder_temporal_strides: List[int] = None  # Thêm cấu hình strides
    ) -> None:
        super(STSE, self).__init__()
        # Store parameters
        self.input_dim = c_in # c_in là số channels đầu vào (ví dụ: 3 cho XYZ)
        self.hidden_dimension = h_dim # h_dim là số channels output của Encoder GAT-TCN stack
        self.latent_dim = latent_dim # latent_dim là kích thước của vector Z sau bottleneck
        self.n_frames = n_frames
        self.n_joints = n_joints
        self.layer_channels_config = layer_channels # Channels cho các GAT_TCN_Block trung gian
        self.dropout = dropout
        self.device = device
        
        # Load adjacency matrix if not provided
        if adjacency_matrix is None:
            try:
                from models.gcae.graph import Graph # Import Graph ở đây
                # Cấu hình layout và strategy có thể lấy từ config tổng của DCMD
                graph_layout = 'ntu-rgb+d' if n_joints == 25 else ('h36m' if n_joints == 17 else 'coco')
                print(f"STSE: Adjacency matrix not provided. Initializing Graph with layout='{graph_layout}'.")
                graph = Graph(layout=graph_layout, strategy='uniform', normalize=True)
                # graph.A là một numpy array
                self.adjacency_matrix = torch.from_numpy(graph.A).float().to(self.device)
            except ImportError:
                print("Warning: models.gcae.graph.Graph not found. Using fallback adjacency matrix.")
                # Fallback: create a simple adjacency matrix
                adj = torch.zeros(n_joints, n_joints, dtype=torch.float32)
                # Example: connect each joint to the next for simplicity, and self-loops
                for i in range(n_joints -1):
                    adj[i, i+1] = 1.0
                    adj[i+1, i] = 1.0
                adj.fill_diagonal_(1.0)
                if n_joints > 0: # Normalize
                    row_sums = adj.sum(dim=1, keepdim=True)
                    self.adjacency_matrix = (adj / (row_sums + 1e-6)).to(self.device) # Add epsilon
                else:
                    self.adjacency_matrix = adj.to(self.device)

        elif isinstance(adjacency_matrix, np.ndarray):
            self.adjacency_matrix = torch.from_numpy(adjacency_matrix).float().to(self.device)
        elif isinstance(adjacency_matrix, torch.Tensor):
            self.adjacency_matrix = adjacency_matrix.to(self.device)
        else:
            raise TypeError("adjacency_matrix must be a NumPy array or a PyTorch tensor.")

        self.num_heads_gat = num_heads_gat

        # Xác định temporal strides cho Encoder
        # Encoder trong components.py sẽ có len(layer_channels_config) + 1 blocks
        num_encoder_blocks = len(self.layer_channels_config) + 1
        if encoder_temporal_strides is None:
            self.encoder_strides = [1] * num_encoder_blocks
            # Áp dụng stride 2 một cách hợp lý, ví dụ:
            if num_encoder_blocks >= 2: # Cần ít nhất 2 block để có stride 2
                self.encoder_strides[1] = 2 # Downsample ở block thứ 2
            if num_encoder_blocks >= 4: # Thêm một lần downsample nữa nếu có đủ block
                self.encoder_strides[3] = 2 # Downsample ở block thứ 4
            # Đảm bảo stride cuối cùng không làm mất quá nhiều thông tin thời gian nếu không muốn
            # Ví dụ: if num_encoder_blocks > 1 and self.encoder_strides[-1] == 2 and num_encoder_blocks % 2 != 0 :
            # self.encoder_strides[-1] = 1 #  Không downsample ở block cuối nếu số block lẻ và block trước đó đã downsample
        elif len(encoder_temporal_strides) == num_encoder_blocks:
            self.encoder_strides = encoder_temporal_strides
        else:
            raise ValueError(f"Length of encoder_temporal_strides ({len(encoder_temporal_strides)}) "
                             f"must match the number of encoder blocks ({num_encoder_blocks}).")

        # Xác định temporal strides cho Decoder
        # Decoder trong components.py sẽ có len(layer_channels_config) + 1 blocks (khi tính cả output_dim)
        num_decoder_blocks = len(self.layer_channels_config) + 1
        if decoder_temporal_strides is None:
            # GAT_TCN_Block với Conv2d không hỗ trợ upsampling bằng stride > 1
            self.decoder_strides = [1] * num_decoder_blocks
        elif len(decoder_temporal_strides) == num_decoder_blocks:
            self.decoder_strides = decoder_temporal_strides
            if any(s > 1 for s in self.decoder_strides):
                print("Warning: Decoder GAT_TCN_Block uses Conv2d; temporal_stride > 1 will not upsample time dimension.")
        else:
            raise ValueError(f"Length of decoder_temporal_strides ({len(decoder_temporal_strides)}) "
                             f"must match the number of decoder blocks ({num_decoder_blocks}).")

        # Build components
        self.build_model()

    def build_model(self) -> None:
        # Encoder
        self.encoder = Encoder(
            input_dim=self.input_dim, # Số channels đầu vào (ví dụ: 3 cho XYZ)
            layer_channels=self.layer_channels_config, # List các output channels của các GAT_TCN_Block trung gian
            hidden_dimension=self.hidden_dimension, # Output channels của GAT_TCN_Block cuối cùng trong encoder
            n_frames=self.n_frames, # Thông tin, không dùng trực tiếp trong GAT_TCN_Block
            n_joints=self.n_joints, # Thông tin, không dùng trực tiếp trong GAT_TCN_Block
            dropout=self.dropout,
            adjacency_matrix=self.adjacency_matrix, # Tensor (V,V) hoặc (num_strat, V,V)
            num_heads_gat=self.num_heads_gat,
            temporal_strides=self.encoder_strides # Truyền strides đã xác định
        )
        
        # Bottleneck to latent space Z
        # Input features cho bottleneck phụ thuộc vào output shape của encoder GAT_TCN stack
        # Encoder output: (N*M, self.hidden_dimension, T_final, V)
        # T_final phụ thuộc vào số lần downsample bằng strides
        # Cần tính T_final một cách linh động
        
        # Calculate T_final after encoder
        current_t = self.n_frames # Start with original number of frames
        # conv_kernel_t = 9 # from TemporalConvNet in gat.py
        # conv_pad_t = (conv_kernel_t - 1) // 2 # = 4

        for s_enc in self.encoder_strides:
            # Correct formula for Conv2d output dimension used in TemporalConvNet
            # L_out = floor((L_in - 1)/stride + 1)
            if current_t >= 1 and s_enc >=1 : # Ensure valid inputs for formula
                 current_t = np.floor((current_t - 1) / s_enc).astype(int) + 1
            else:
                 # Handle edge cases or raise error if T becomes too small unexpectedly
                 print(f"Warning: Unexpected current_t ({current_t}) or s_enc ({s_enc}) during T_final calculation.")
                 # Fallback to ensure T is at least 1, though this might hide issues
                 current_t = max(1, current_t)


        self.t_final_encoder = current_t # This is the *calculated* T after all strides in encoder
        
        in_features_bottleneck = self.hidden_dimension * self.t_final_encoder * self.n_joints
        self.btlnk = nn.Linear(in_features=in_features_bottleneck, out_features=self.latent_dim)

    def encode(
        self,
        X: torch.Tensor, # Input shape: (batch, c_in, n_frames, n_joints) hoặc (batch, n_frames, n_joints, c_in)
        return_shape: bool = False,
        t_diffusion: torch.Tensor = None # Tham số t cho diffusion (nếu cần, hiện tại GAT_TCN_Block không dùng)
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Size]]:
        
        # Xử lý input shape: mong muốn (N, C, T, V) cho Encoder
        if X.dim() == 4 and X.size(1) == self.input_dim : # (N, C, T, V)
            N, C, T, V = X.shape
            X_proc = X
            num_persons = 1 # Giả định 1 người nếu không có chiều M
        elif X.dim() == 4 and X.size(3) == self.input_dim: # (N, T, V, C)
            N, T, V, C = X.shape
            X_proc = X.permute(0, 3, 1, 2).contiguous() # (N, C, T, V)
            num_persons = 1
        elif X.dim() == 5 and X.size(1) == self.input_dim : # (N, C, T, V, M) - ít phổ biến hơn
            N, C, T, V, M_persons = X.size()
            X_proc = X.permute(0, 4, 1, 2, 3).contiguous().view(N * M_persons, C, T, V)
            num_persons = M_persons # Lưu lại để reshape nếu cần
        elif X.dim() == 5 and X.size(2) == self.input_dim : # (N, M, C, T, V) - DCMD hay dùng
            N, M_persons, C, T, V = X.size()
            X_proc = X.view(N * M_persons, C, T, V)
            num_persons = M_persons
        else:
            raise ValueError(f"Input X has an unexpected shape: {X.shape}. "
                             f"Expected (N,C,T,V), (N,T,V,C), (N,M,C,T,V) or (N,C,T,V,M) with C={self.input_dim}")

        # Apply encoder
        # X_proc shape: (N_actual, C_in, T_in, V)
        encoded_features, _residuals = self.encoder(X_proc, t_diffusion) # _residuals không dùng trong STSE cơ bản
        # encoded_features shape: (N_actual, self.hidden_dimension, T_final_encoder, V)
        
        X_shape_after_gat_stack = encoded_features.size() 
        
        # Flatten and bottleneck
        # Reshape để phù hợp với self.btlnk đã được tính toán với t_final_encoder
        if X_shape_after_gat_stack[2] != self.t_final_encoder or X_shape_after_gat_stack[3] != self.n_joints:
             print(f"Warning: Encoder output T,V ({X_shape_after_gat_stack[2]},{X_shape_after_gat_stack[3]}) "
                   f"differs from expected ({self.t_final_encoder},{self.n_joints}). Check strides and n_joints.")

        flattened_features = encoded_features.reshape(X_shape_after_gat_stack[0], -1) # (N_actual, self.hidden_dimension * T_final_encoder * V)
        Z = self.btlnk(flattened_features) # (N_actual, self.latent_dim)
        
        if return_shape:
            return Z, X_shape_after_gat_stack # Trả về shape trước khi flatten cho Decoder
        return Z

    def forward(self, X: torch.Tensor, t_diffusion: torch.Tensor = None) -> Tuple[torch.Tensor, None]:
        # STSE chỉ làm nhiệm vụ encode, không decode. Output Z và None (cho X_rec)
        Z = self.encode(X, return_shape=False, t_diffusion=t_diffusion)
        return Z, None # Trả về Z và None để khớp với output signature của STSAE nếu cần

class STSAE(STSE): # Spatio-Temporal SpGraphAutoEncoder
    def __init__(
        self,
        c_in: int,
        h_dim: int = 32,
        latent_dim: int = 64,
        n_frames: int = 12,
        n_joints: int = 17,
        layer_channels: List[int] = [64, 128, 256], # Shared for encoder/decoder structure
        dropout: float = 0.3,
        adjacency_matrix=None,
        num_heads_gat: int = 4,
        encoder_temporal_strides: List[int] = None,
        decoder_temporal_strides: List[int] = None,
        device: Union[str, torch.device] = 'cpu' # Sửa DeviceObjType
    ) -> None:
        super(STSAE, self).__init__(
            c_in=c_in,
            h_dim=h_dim,
            latent_dim=latent_dim,
            n_frames=n_frames,
            n_joints=n_joints,
            layer_channels=layer_channels, # Sẽ được dùng để cấu hình encoder_strides
            dropout=dropout,
            device=device,
            adjacency_matrix=adjacency_matrix,
            num_heads_gat=num_heads_gat,
            encoder_temporal_strides=encoder_temporal_strides, # Truyền xuống super
            decoder_temporal_strides=decoder_temporal_strides  # Truyền xuống super (sẽ dùng trong build_decoder)
        )
        # Decoder
        self.build_decoder()

    def build_decoder(self) -> None:
        # output_dim của Decoder là self.input_dim (c_in ban đầu)
        # layer_channels cho Decoder sẽ đảo ngược của Encoder (không tính h_dim và c_in)
        # hidden_dimension đầu vào cho Decoder là self.hidden_dimension (output của Encoder GAT_TCN stack)
        
        self.decoder = Decoder(
            output_dim=self.input_dim, # Target C_out là c_in ban đầu
            layer_channels=self.layer_channels_config, # List các channels trung gian (sẽ được đảo ngược trong Decoder)
            hidden_dimension=self.hidden_dimension, # Input C_in cho Decoder GAT_TCN stack
            n_frames=self.n_frames, # Thông tin, không dùng trực tiếp (T_final_encoder là T đầu vào cho Decoder)
            n_joints=self.n_joints, # Thông tin
            dropout=self.dropout,
            adjacency_matrix=self.adjacency_matrix,
            num_heads_gat=self.num_heads_gat,
            temporal_strides=self.decoder_strides # Strides đã được xác định trong __init__ của STSE
        )
        
        # Reverse bottleneck
        # Input cho rev_btlnk là self.latent_dim
        # Output features phải khớp với input của Decoder GAT_TCN stack trước khi reshape
        # đó là self.hidden_dimension * self.t_final_encoder * self.n_joints
        out_features_rev_bottleneck = self.hidden_dimension * self.t_final_encoder * self.n_joints
        self.rev_btlnk = nn.Linear(in_features=self.latent_dim, out_features=out_features_rev_bottleneck)

    def decode(self, Z: torch.Tensor, shape_before_flatten: Tuple[int], t_diffusion: torch.Tensor = None) -> torch.Tensor:
        # Z: (N_actual, self.latent_dim)
        # shape_before_flatten: (N_actual, self.hidden_dimension, T_final_encoder, V)
        
        X_from_latent = self.rev_btlnk(Z) # (N_actual, self.hidden_dimension * T_final_encoder * V)
        
        # Reshape để khớp với input của Decoder GAT_TCN stack
        # (N_actual, self.hidden_dimension, self.t_final_encoder, self.n_joints)
        reconstructed_spatial_temporal_shape = X_from_latent.view(shape_before_flatten)
        
        # Apply decoder
        # X_rec shape: (N_actual, self.input_dim, T_output_decoder, V)
        # T_output_decoder sẽ bằng T_final_encoder nếu decoder_strides toàn là 1
        X_rec = self.decoder(reconstructed_spatial_temporal_shape, t_diffusion)
        
        # Xử lý output shape để trả về (N, C, T, V) hoặc (N, M, C, T, V)
        # Hiện tại X_rec là (N_actual, self.input_dim, T_out_dec, V)
        # Nếu num_persons được lưu từ encode, có thể reshape lại.
        # Tuy nhiên, DCMD thường xử lý (N*M) ở ngoài rồi reshape lại.
        return X_rec

    def forward(
        self,
        X: torch.Tensor, # Input shape: (batch, c_in, n_frames, n_joints) hoặc (N,M,C,T,V)
        t_diffusion: torch.Tensor = None
    ) -> Tuple[torch.Tensor, torch.Tensor]: # Z, X_reconstructed
        
        original_shape = X.shape
        M_persons = 1 # CORRECTED LINE
        if X.dim() == 5 and X.size(1) != self.input_dim: # (N, M, C, T, V) - Assumes C is at index 2
            if X.size(2) == self.input_dim: # Check if C is indeed at index 2
                 M_persons = original_shape[1]
        elif X.dim() == 5 and X.size(1) == self.input_dim: # (N, C, T, V, M) - Assumes M is at index 4
             if X.size(4) != self.n_joints : # A simple check if last dim could be M
                  M_persons = original_shape[4]


        Z, shape_before_flatten = self.encode(X, return_shape=True, t_diffusion=t_diffusion)
        X_rec = self.decode(Z, shape_before_flatten, t_diffusion=t_diffusion)

        # Reshape X_rec về lại dạng có thể có M (num_persons) nếu input ban đầu có
        # X_rec hiện là (N_actual, self.input_dim, T_out_decoder, V)
        # N_actual = N * M_persons
        if M_persons > 1 and X.dim() == 5:
            if original_shape[1] == M_persons : # (N, M, C, T, V)
                 N = original_shape[0]
                 C_out, T_out, V_out = X_rec.shape[1], X_rec.shape[2], X_rec.shape[3]
                 X_rec = X_rec.view(N, M_persons, C_out, T_out, V_out)
            elif original_shape[4] == M_persons : # (N, C, T, V, M)
                 N, C_out, T_in_orig, V_in_orig = original_shape[0], original_shape[1], original_shape[2], original_shape[3]
                 _, _, T_out, V_out = X_rec.shape
                 X_rec = X_rec.view(N, C_out, T_out, V_out, M_persons)

        return Z, X_rec

if __name__ == '__main__':
    # --- Example Usage ---
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Parameters
    c_in_test = 3
    n_frames_test = 24
    n_joints_test = 17 # Cho Human3D (h36m layout)
    #n_joints_test = 25 # Cho NTU-RGB+D

    batch_size_test = 2
    num_persons_test = 1 # Hoặc 2 để test trường hợp M > 1

    # Model configuration
    h_dim_test = 64         # Output channels of encoder GAT stack
    latent_dim_test = 128   # Bottleneck Z dimension
    # layer_channels_test = [64, 128, 256] # 3 intermediate GAT_TCN blocks in encoder before final projection to h_dim
                                        # Encoder sẽ có 3+1 = 4 GAT_TCN blocks
    layer_channels_test = [c_in_test*2, c_in_test*4, c_in_test*8] # Ví dụ: [6, 12, 24]
    num_encoder_gat_tcn_blocks = len(layer_channels_test) + 1 # +1 vì block cuối ra h_dim_test
    
    # Encoder strides: 4 blocks -> [1,2,1,2] -> T_out = T_in / 4
    # enc_strides = [1, 2, 1, 2] if num_encoder_gat_tcn_blocks == 4 else [1]*num_encoder_gat_tcn_blocks
    enc_strides = [1] * num_encoder_gat_tcn_blocks
    if num_encoder_gat_tcn_blocks >=2 : enc_strides[1] = 2
    if num_encoder_gat_tcn_blocks >=4 : enc_strides[3] = 2


    # Decoder strides: 4 blocks (input h_dim_test -> intermediate -> ... -> c_in_test) -> [1,1,1,1]
    num_decoder_gat_tcn_blocks = len(layer_channels_test) + 1
    # dec_strides = [1, 1, 1, 1] if num_decoder_gat_tcn_blocks == 4 else [1]*num_decoder_gat_tcn_blocks
    dec_strides = [1] * num_decoder_gat_tcn_blocks
    # If GAT_TCN_Block supported ConvTranspose, you might do:
    # if num_decoder_gat_tcn_blocks >=2 : dec_strides[0] = 2 # Upsample at start
    # if num_decoder_gat_tcn_blocks >=4 : dec_strides[2] = 2 # Upsample again


    print(f"Configuring STSAE with: \n"
          f"  Input Channels (c_in): {c_in_test}\n"
          f"  Encoder GAT Stack Output Channels (h_dim): {h_dim_test}\n"
          f"  Bottleneck Latent Dim (latent_dim): {latent_dim_test}\n"
          f"  Frames: {n_frames_test}, Joints: {n_joints_test}\n"
          f"  Intermediate Layer Channels Config: {layer_channels_test}\n"
          f"  Encoder Temporal Strides: {enc_strides}\n"
          f"  Decoder Temporal Strides: {dec_strides}")


    # Create STSAE model instance
    stsae_model = STSAE(
        c_in=c_in_test,
        h_dim=h_dim_test,
        latent_dim=latent_dim_test,
        n_frames=n_frames_test,
        n_joints=n_joints_test,
        layer_channels=layer_channels_test,
        dropout=0.1,
        adjacency_matrix=None, # Let it use Graph class based on n_joints
        num_heads_gat=2,
        encoder_temporal_strides=enc_strides,
        decoder_temporal_strides=dec_strides,
        device=device
    ).to(device)

    # Create dummy input tensor
    # DCMD thường dùng (N, M, C, T, V)
    if num_persons_test > 1:
        dummy_X = torch.randn(batch_size_test, num_persons_test, c_in_test, n_frames_test, n_joints_test).to(device)
    else: # (N, C, T, V)
        dummy_X = torch.randn(batch_size_test, c_in_test, n_frames_test, n_joints_test).to(device)

    print(f"\nInput X shape: {dummy_X.shape}")

    # Test forward pass
    try:
        Z_latent, X_reconstructed = stsae_model(dummy_X)
        print(f"Latent Z shape: {Z_latent.shape}") # Expected: (batch_size * num_persons, latent_dim)
        print(f"Reconstructed X_rec shape: {X_reconstructed.shape}") # Expected: same as processed input to encoder
        
        # Check if output shape matches input shape (considering num_persons)
        expected_rec_shape = dummy_X.shape
        if num_persons_test > 1 and dummy_X.dim() == 5 and dummy_X.size(1) == num_persons_test: # N,M,C,T,V
            pass # X_reconstructed should already be in N,M,C,T,V
        elif num_persons_test == 1 and dummy_X.dim() == 4: # N,C,T,V
             pass # X_reconstructed should already be in N,C,T,V
        else: # Other cases might require specific reshape check not covered by current auto-reshape in STSAE forward
            print(f"Note: Detailed output shape check for M>1 might need more specific logic if input was not N,M,C,T,V.")


        assert X_reconstructed.shape == expected_rec_shape, \
            f"Shape mismatch! Reconstructed: {X_reconstructed.shape}, Expected: {expected_rec_shape}"
        print("Forward pass successful and output shape matches input shape.")

    except Exception as e:
        print(f"Error during STSAE forward pass: {e}")
        import traceback
        traceback.print_exc()
        
    # Print number of parameters
    num_params = sum(p.numel() for p in stsae_model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters in STSAE model: {num_params:,}")

Overwriting /kaggle/working/DCMD-main/models/stsae/stsae.py


In [12]:
%%writefile /kaggle/working/DCMD-main/models/gcae/gat.py

# models/gcae/gat.py (hoặc một đường dẫn tương tự trong dự án của bạn)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from torch_geometric.utils import dense_to_sparse # Tiện ích chuyển A sang edge_index

# Giả sử bạn có một lớp Graph tương tự như trong project gốc để lấy ma trận kề
# Ví dụ: from .graph import Graph 
# Nếu không, bạn cần truyền ma trận kề A trực tiếp vào Model_GAT

# --- Utility Function ---
def adj_to_edge_index(adj_matrix):
    """
    Chuyển một ma trận kề (dense) sang định dạng edge_index của PyG.
    Args:
        adj_matrix (torch.Tensor): Ma trận kề, shape (num_nodes, num_nodes)
    Returns:
        torch.Tensor: edge_index, shape (2, num_edges)
    """
    edge_index, _ = dense_to_sparse(adj_matrix)
    return edge_index

# --- Spatial Attention Layer (GAT) ---
class SpatialAttentionLayer(nn.Module):
    def __init__(self, in_channels, out_channels, num_heads, concat=True, dropout=0.6, leaky_relu_negative_slope=0.2):
        super().__init__()
        self.gat_conv = GATConv(in_channels, 
                                out_channels // num_heads if concat else out_channels,
                                heads=num_heads, 
                                concat=concat, 
                                dropout=dropout,
                                negative_slope=leaky_relu_negative_slope)
        self.concat = concat
        self.num_heads = num_heads
        self.out_channels_per_head = out_channels // num_heads if concat else out_channels

    def forward(self, x_node_features, edge_index):
        """
        Args:
            x_node_features (torch.Tensor): Features của các node. 
                                           Shape: (batch_size * num_nodes, in_channels)
                                           hoặc (num_nodes, in_channels) nếu xử lý từng sample.
            edge_index (torch.Tensor): Định nghĩa kết nối đồ thị. Shape: (2, num_edges)
        Returns:
            torch.Tensor: Output features sau GAT. Shape: (batch_size * num_nodes, out_channels)
        """
        return self.gat_conv(x_node_features, edge_index)

# --- Temporal Convolutional Network (TCN) Block ---
class TemporalConvNet(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=9, stride=1, dropout=0.2):
        super().__init__()
        pad = (kernel_size - 1) // 2
        self.tcn_block = nn.Sequential(
            nn.BatchNorm2d(in_channels), # Batch norm trên channels
            nn.ReLU(inplace=True),
            nn.Conv2d(
                in_channels,
                out_channels,
                (kernel_size, 1), # Kernel chỉ theo chiều thời gian T, không gian là 1
                (stride, 1),      # Stride chỉ theo chiều thời gian T
                (pad, 0),         # Padding chỉ theo chiều thời gian T
            ),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout, inplace=True) # Dropout sau batch norm và conv
        )

    def forward(self, x):
        """
        Args:
            x (torch.Tensor): Input tensor. Shape: (N, C_in, T, V) 
                              N: batch_size, C_in: channels, T: time_frames, V: num_nodes (vertices)
        Returns:
            torch.Tensor: Output tensor. Shape: (N, C_out, T_out, V)
        """
        return self.tcn_block(x)

# --- GAT-TCN Block ---
class GAT_TCN_Block(nn.Module):
    def __init__(self, in_channels, out_channels, num_heads_gat, 
                 temporal_kernel_size=9, temporal_stride=1,
                 residual=True, dropout=0.2, A_is_dense=True):
        super().__init__()
        self.residual = residual
        self.A_is_dense = A_is_dense # True nếu A là ma trận kề, False nếu A đã là edge_index

        # Spatial Attention Layer
        # GAT sẽ hoạt động trên từng frame, nên input channels là in_channels
        # Output channels của GAT sẽ là input channels cho TCN
        self.spatial_attn = SpatialAttentionLayer(in_channels, out_channels, num_heads_gat, dropout=dropout)
        
        # Temporal Convolutional Layer
        # TCN hoạt động trên output của GAT. Channels đầu vào của TCN là out_channels từ GAT.
        self.tcn = TemporalConvNet(out_channels, out_channels, 
                                   kernel_size=temporal_kernel_size, 
                                   stride=temporal_stride, 
                                   dropout=dropout)

        if not residual:
            self.residual_conv = lambda x: 0
        elif (in_channels == out_channels) and (temporal_stride == 1):
            self.residual_conv = lambda x: x
        else:
            # Conv2d để điều chỉnh số kênh và kích thước thời gian cho residual connection
            self.residual_conv = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(temporal_stride, 1)),
                nn.BatchNorm2d(out_channels),
            )
        
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, adj):
        """
        Args:
            x (torch.Tensor): Input tensor. Shape: (N, C_in, T, V)
            adj (torch.Tensor): Ma trận kề (nếu self.A_is_dense=True, shape (V,V) hoặc (num_adj_matrices, V,V))
                               hoặc edge_index (nếu self.A_is_dense=False, shape (2, num_edges))
        Returns:
            torch.Tensor: Output tensor. Shape: (N, C_out, T_out, V)
        """
        N, C_in, T, V = x.size()
        res = self.residual_conv(x) # (N, C_out, T_out, V)

        # --- Spatial Processing (GAT on each time frame) ---
        # GATConv thường mong muốn input (num_total_nodes, num_node_features)
        # Ta sẽ xử lý từng frame thời gian hoặc batch tất cả các node qua GAT
        
        # Chuyển x về dạng (N, T, V, C_in) để dễ lặp qua T
        x_permuted = x.permute(0, 2, 3, 1).contiguous() # (N, T, V, C_in)
        
        batched_node_features = x_permuted.view(N * T * V, C_in) # (N*T*V, C_in)

        # Xử lý edge_index:
        if self.A_is_dense:
            if adj.dim() == 3: # (num_adj_strategies, V, V), chọn 1 strategy
                current_adj = adj[0] # Ví dụ: chọn strategy đầu tiên
            elif adj.dim() == 2: # (V,V)
                current_adj = adj
            else:
                raise ValueError(f"Adjacency matrix `adj` has unexpected dimensions: {adj.shape}")
            
            # Tạo edge_index cho một đồ thị (V,V)
            single_graph_edge_index = adj_to_edge_index(current_adj).to(x.device)
            
            # Mở rộng edge_index cho N*T batches của đồ thị
            # Mỗi đồ thị trong batch (N*T cái) có V nodes
            # Đây là phần quan trọng và có thể cần tối ưu hóa
            # Cách đơn giản: lặp (chậm) hoặc dùng các hàm batching của PyG (hiệu quả hơn)
            
            # Cách hiệu quả hơn: Tạo edge_index cho batch các đồ thị giống hệt nhau
            num_graphs_in_batch = N * T
            node_offsets = torch.arange(0, num_graphs_in_batch * V, step=V, device=x.device) # (N*T)
            
            # Mở rộng edge_index
            batch_edge_index_0 = single_graph_edge_index[0].repeat(num_graphs_in_batch) + \
                                 node_offsets.repeat_interleave(single_graph_edge_index.size(1))
            batch_edge_index_1 = single_graph_edge_index[1].repeat(num_graphs_in_batch) + \
                                 node_offsets.repeat_interleave(single_graph_edge_index.size(1))
            batched_edge_index = torch.stack([batch_edge_index_0, batch_edge_index_1], dim=0) # (2, (N*T)*num_edges_single)

        else: # adj đã là edge_index (giả sử nó đã được batch đúng cách hoặc là cố định)
            if adj.size(0) != 2 :
                 raise ValueError(f"Provided edge_index `adj` has unexpected shape: {adj.shape}")
            # Nếu adj là edge_index cho 1 đồ thị, cần batch nó tương tự như trên
            # Giả sử adj đã được batch sẵn cho N*T đồ thị
            batched_edge_index = adj.to(x.device) 
            # Lưu ý: Nếu adj chỉ là edge_index của một đồ thị đơn lẻ, bạn cần logic batching ở đây.

        # Apply GAT
        # batched_node_features: (N*T*V, C_in)
        # batched_edge_index: (2, (N*T)*num_edges_single)
        gat_out_flat = self.spatial_attn(batched_node_features, batched_edge_index) # (N*T*V, C_out)
        
        # Reshape lại về (N, T, V, C_out)
        x_spatial_processed = gat_out_flat.view(N, T, V, -1) # C_out sẽ được suy ra
        
        # Chuyển về (N, C_out, T, V) cho TCN
        x_for_tcn = x_spatial_processed.permute(0, 3, 1, 2).contiguous()

        # --- Temporal Processing (TCN) ---
        x_temporal = self.tcn(x_for_tcn) # (N, C_out, T_out, V)
        
        # Residual connection and ReLU
        x_out = self.relu(x_temporal + res)
        return x_out

# --- Main GAT-based Model ---
class Model_GAT(nn.Module):
    def __init__(
            self,
            in_channels,           # non-default
            num_classes,           # non-default
            num_nodes,             # non-default
            adjacency_matrix,      # non-default
            num_heads_gat=8,       # default
            dropout=0.2,           # default
            gat_tcn_channels_config=None,  # default
            **kwargs               # default-capturing
        ):
        super().__init__()
        self.A_is_dense = True

        if gat_tcn_channels_config is None:
            # Cấu hình mặc định tương tự STGCN gốc cho dễ so sánh
            gat_tcn_channels_config = [
                (in_channels, 64, 1), (64, 64, 1), (64, 64, 1),
                (64, 128, 2), (128, 128, 1), (128, 128, 1),
                (128, 256, 2), (256, 256, 1), (256, 256, 1)
            ]

        # Lưu trữ ma trận kề (hoặc edge_index nếu đã xử lý trước)
        # Giả sử adjacency_matrix là ma trận kề (dense)
        # Nếu A có nhiều "strategies" như trong STGCN, ta sẽ chọn 1 hoặc cần logic xử lý phức tạp hơn
        if isinstance(adjacency_matrix, list) or \
           (isinstance(adjacency_matrix, torch.Tensor) and adjacency_matrix.dim() == 3):
            # Giả sử dùng strategy đầu tiên nếu có nhiều
            self.register_buffer('A', torch.tensor(adjacency_matrix[0], dtype=torch.float32) 
                                     if isinstance(adjacency_matrix, list) else adjacency_matrix[0].float())
        elif isinstance(adjacency_matrix, torch.Tensor) and adjacency_matrix.dim() == 2:
            self.register_buffer('A', adjacency_matrix.float())
        else:
            raise ValueError("adjacency_matrix format not recognized or not provided.")

        self.data_bn = nn.BatchNorm1d(in_channels * num_nodes) # Chuẩn hóa ban đầu

        self.gat_tcn_blocks = nn.ModuleList()
        current_channels = in_channels
        for i, (block_in_c, block_out_c, block_stride_t) in enumerate(gat_tcn_channels_config):
            # Đảm bảo in_channels của block khớp với current_channels
            actual_block_in_c = current_channels if i==0 else gat_tcn_channels_config[i-1][1]

            self.gat_tcn_blocks.append(
                GAT_TCN_Block(
                    in_channels=actual_block_in_c,
                    out_channels=block_out_c,
                    num_heads_gat=num_heads_gat,
                    temporal_stride=block_stride_t,
                    dropout=dropout,
                    A_is_dense=True # Cho biết self.A là ma trận kề
                )
            )
            current_channels = block_out_c # Cập nhật cho block tiếp theo (ít quan trọng nếu config rõ ràng)

        # Output layer (ví dụ: cho classification hoặc feature extraction)
        # Kích thước này phụ thuộc vào nhiệm vụ cuối cùng của DCMD
        # Ví dụ: một lớp Fully Connected sau global average pooling
        self.pool = nn.AdaptiveAvgPool2d((1, 1)) # Pool qua T và V
        self.fc = nn.Linear(current_channels, num_classes)


    def forward(self, x):
        # Input x shape: (N, C, T, V, M) hoặc (N, C, T, V)
        # N: batch_size, C: channels (e.g., 3 for xyz), T: frames, V: joints, M: people
        if x.dim() == 5: # N, C, T, V, M
            N, C, T, V, M = x.size()
            x = x.permute(0, 4, 1, 2, 3).contiguous()  # N, M, C, T, V
            x = x.view(N * M, C, T, V) # (N*M, C, T, V)
        elif x.dim() == 4: # (N, C, T, V)
            N_actual, C, T, V = x.size() # N_actual có thể là N*M
        else:
            raise ValueError(f"Input tensor x has unexpected dimensions: {x.shape}")

        # Data BN: (N*M, C, T, V) -> (N*M, C*V, T) -> BN -> (N*M, C, T, V)
        x_bn_in = x.permute(0, 1, 3, 2).contiguous().view(x.size(0), C * V, T)
        x_bn_out = self.data_bn(x_bn_in)
        x = x_bn_out.view(x.size(0), C, V, T).permute(0, 1, 3, 2).contiguous() # (N*M, C, T, V)

        # GAT-TCN Blocks
        for block in self.gat_tcn_blocks:
            x = block(x, self.A) # Truyền ma trận kề A (hoặc edge_index đã xử lý)

        # Global Pooling và FC (ví dụ cho classification)
        # x shape sau blocks: (N*M, C_out, T_final, V)
        pooled_x = self.pool(x) # (N*M, C_out, 1, 1)
        squeezed_x = pooled_x.view(pooled_x.size(0), -1) # (N*M, C_out)
        
        out = self.fc(squeezed_x) # (N*M, num_classes)

        # Reshape lại nếu ban đầu có M (số người)
        if x.dim() == 5 and M > 1: # Nếu input gốc có M
             out = out.view(N, M, -1)

        return out

# --- Ví dụ cách sử dụng (để kiểm tra nhanh, không phải để huấn luyện) ---
if __name__ == '__main__':
    # --- Thông số giả định ---
    batch_size = 2
    in_channels_data = 3  # xyz
    num_frames = 32
    num_nodes_data = 25 # Số khớp
    num_persons = 1 
    num_classes_out = 10 # Ví dụ cho classification

    # Tạo ma trận kề giả định (ví dụ: đồ thị kết nối đầy đủ đơn giản)
    # Trong thực tế, bạn sẽ load từ graph.py hoặc file cấu hình
    adj = torch.ones(num_nodes_data, num_nodes_data) # Kết nối đầy đủ (bao gồm self-loops)
    # adj = torch.rand(num_nodes_data, num_nodes_data) > 0.7 # Đồ thị thưa hơn
    # adj.fill_diagonal_(1) # Đảm bảo self-loops

    # Khởi tạo model
    gat_model = Model_GAT(
        in_channels=in_channels_data,
        num_classes=num_classes_out,
        num_nodes=num_nodes_data,
        adjacency_matrix=adj, # Truyền ma trận kề
        num_heads_gat=4,
        dropout=0.25,
        # Bỏ qua gat_tcn_channels_config để dùng mặc định
    )

    # Tạo dữ liệu đầu vào giả
    # (N, C, T, V) hoặc (N, C, T, V, M)
    if num_persons > 1:
        dummy_input = torch.randn(batch_size, in_channels_data, num_frames, num_nodes_data, num_persons)
    else:
        dummy_input = torch.randn(batch_size, in_channels_data, num_frames, num_nodes_data)
    
    # Đặt model ở chế độ eval để dropout không hoạt động (nếu có)
    gat_model.eval() 
    
    # Forward pass
    try:
        print(f"Input shape: {dummy_input.shape}")
        output = gat_model(dummy_input)
        print(f"Output shape: {output.shape}") # Mong muốn (batch_size * num_persons, num_classes_out)
                                            # Hoặc (batch_size, num_persons, num_classes_out) nếu M > 1
    except Exception as e:
        print(f"Error during forward pass: {e}")
        import traceback
        traceback.print_exc()

    # Kiểm tra số lượng tham số
    num_params = sum(p.numel() for p in gat_model.parameters() if p.requires_grad)
    print(f"Number of trainable parameters in Model_GAT: {num_params:,}")

    # --- Ví dụ với edge_index đã được chuẩn bị trước (ít phổ biến hơn khi dùng trực tiếp A) ---
    # print("\n--- Example with pre-computed edge_index (less common for this setup) ---")
    # single_edge_index = adj_to_edge_index(adj) 
    # gat_model_with_edge_idx = Model_GAT(...) # Cần sửa Model_GAT để nhận is_A_dense=False và edge_index
    # ...

Writing /kaggle/working/DCMD-main/models/gcae/gat.py


In [20]:
%%writefile /kaggle/working/DCMD-main/models/common/components.py
from typing import List, Tuple, Union

import torch
import torch.nn as nn
# import models.gcae.stsgcn as stsgcn  # Keep for legacy if needed, or remove if fully transitioned
# Ensure GAT_TCN_Block is correctly imported from your gat.py (assuming it's in models.gcae.gat)
from models.gcae.gat import GAT_TCN_Block
# Ensure Graph is imported from your new graph.py
from models.gcae.graph import Graph # This import should now work

class Encoder(nn.Module):
    def __init__(
            self,
            input_dim: int,
            layer_channels: List[int],
            hidden_dimension: int, # This is the output dim of the last GAT_TCN_Block
            n_frames: int, # Currently not directly used by GAT_TCN_Block layers
            n_joints: int, # Currently not directly used by GAT_TCN_Block layers
            dropout: float,
            adjacency_matrix: torch.Tensor, # Expects a (V,V) or (num_strategies, V,V) tensor
            num_heads_gat: int = 4,
            temporal_strides: List[int] = None, # Add this to control strides per layer
            bias: bool = True
        ) -> None:
        super().__init__()
        self.input_dim = input_dim
        
        # The last channel in layer_channels should be hidden_dimension
        if layer_channels[-1] != hidden_dimension:
            # Or adjust layer_channels to ensure the final output is hidden_dimension
            # For simplicity, let's assume layer_channels defines intermediate outputs
            # and the last one is the input to a final projection if needed,
            # or hidden_dimension is the output of the last GAT_TCN layer.
             # Let's make layer_channels define all block outputs, hidden_dimension is the last one
            if not any(lc == hidden_dimension for lc in layer_channels):
                 # If hidden_dimension is meant to be the final output channel from the GAT_TCN stack
                self.layer_channels = layer_channels + [hidden_dimension]
            else: # Assume hidden_dimension is already the last element of layer_channels
                self.layer_channels = layer_channels

        else: # if layer_channels[-1] == hidden_dimension
            self.layer_channels = layer_channels

        self.internal_block_output_channels = layer_channels + [hidden_dimension]
        self.hidden_dimension = hidden_dimension # Final output channel count
        self.n_frames = n_frames
        self.n_joints = n_joints
        self.dropout = dropout
        self.adjacency_matrix = adjacency_matrix # This should be a tensor
        self.num_heads_gat = num_heads_gat
        self.bias = bias # Bias for GAT_TCN_Block not explicitly used in GATConv/Conv2d like Linear

        if temporal_strides is None:
            self.temporal_strides = [1] * len(self.internal_block_output_channels)
        elif len(temporal_strides) != len(self.internal_block_output_channels):
            raise ValueError(
                f"Length of temporal_strides ({len(temporal_strides)}) must match "
                f"the number of GAT-TCN blocks ({len(self.internal_block_output_channels)}) in Encoder."
            )
        else:
            self.temporal_strides = temporal_strides

        self.build_model()

    def build_model(self):
        input_channels = self.input_dim
        model_layers = nn.ModuleList()
        
        # Ensure adjacency_matrix is a tensor before passing to blocks
        if not isinstance(self.adjacency_matrix, torch.Tensor):
            # Assuming it's a numpy array from Graph class if not already a tensor
            current_adj_matrix = torch.from_numpy(self.adjacency_matrix).float()
        else:
            current_adj_matrix = self.adjacency_matrix

        # If adjacency_matrix has multiple strategies (e.g., (num_strat, V, V))
        # GAT_TCN_Block currently expects one (V,V) matrix if A_is_dense=True.
        # Or it expects a pre-batched edge_index.
        # For simplicity, if multiple A's, use the first one.
        if current_adj_matrix.ndim == 3:
            # print(f"Encoder: Using the first adjacency matrix strategy. Shape: {current_adj_matrix.shape}")
            self.processed_adj_matrix = current_adj_matrix[0]
        elif current_adj_matrix.ndim == 2:
            self.processed_adj_matrix = current_adj_matrix
        else:
            raise ValueError(f"Adjacency matrix has unexpected dimensions: {current_adj_matrix.shape}")


        for i, out_channels in enumerate(self.layer_channels):
            model_layers.append(
                GAT_TCN_Block(
                    in_channels=input_channels,
                    out_channels=out_channels,
                    num_heads_gat=self.num_heads_gat,
                    temporal_kernel_size=9, # Can be made configurable
                    temporal_stride=self.temporal_strides[i], # Use configured stride
                    dropout=self.dropout,
                    A_is_dense=True # Assumes processed_adj_matrix is (V,V)
                )
            )
            input_channels = out_channels
        self.model_layers = model_layers

    def forward(self, X: torch.Tensor, t: torch.Tensor = None) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        # X expected shape: (N_actual, C_in, T, V) where N_actual could be N*M
        layers_out = [X]
        device = X.device # Get device from input tensor
        adj_matrix_on_device = self.processed_adj_matrix.to(device)

        for layer in self.model_layers:
            out_X = layer(layers_out[-1], adj_matrix_on_device)
            layers_out.append(out_X)
        return layers_out[-1], layers_out[:-1]


class Decoder(nn.Module):
    def __init__(
        self,
        output_dim: int, # This is c_in of the original data
        layer_channels: List[int], # Intermediate channels, should be reverse of encoder's intermediate
        hidden_dimension: int, # This is the input dim to the Decoder (output of Encoder's GAT_TCN stack)
        n_frames: int,
        n_joints: int,
        dropout: float,
        adjacency_matrix: torch.Tensor,
        num_heads_gat: int = 4,
        temporal_strides: List[int] = None, # For upsampling, stride usually 1, or use ConvTranspose
        bias: bool = True
    ) -> None:
        super().__init__()
        self.output_dim = output_dim
        # For decoder, layer_channels are typically reversed from encoder,
        # starting from hidden_dimension and ending at output_dim
        self.layer_channels_config = [hidden_dimension] + layer_channels[::-1] # Start with hidden_dim
        # The final GAT_TCN block should output self.output_dim
        self.final_layer_channels = self.layer_channels_config[:-1] # channels for GAT_TCN blocks
        self.final_output_channel = self.layer_channels_config[-1] # target for a final projection
        
        # Let's adjust to make GAT_TCN_Block outputs match layer_channels
        # Input to first GAT_TCN is hidden_dimension
        # Output of last GAT_TCN is output_dim
        self.layer_channels_for_gat_tcn = layer_channels[::-1] + [output_dim]


        self.hidden_dimension = hidden_dimension # Input channel for the first GAT_TCN_Block
        self.n_frames = n_frames
        self.n_joints = n_joints
        self.dropout = dropout
        self.adjacency_matrix = adjacency_matrix
        self.num_heads_gat = num_heads_gat
        self.bias = bias

        if temporal_strides is None:
            # For decoder, strides are often 1, or if upsampling is needed via strided conv,
            # it's usually ConvTranspose2d. GAT_TCN_Block uses Conv2d.
            # So, for upsampling T, GAT_TCN_Block is not ideal as is.
            # Assuming stride 1 for TCN blocks in decoder for now.
            self.temporal_strides = [1] * len(self.layer_channels_for_gat_tcn)
        elif len(temporal_strides) != len(self.layer_channels_for_gat_tcn):
            # If strides are provided, they should match the number of GAT_TCN blocks
            # This might be used if one wants to use "fractionally strided convolutions"
            # conceptually, but Conv2d in TCN doesn't directly do T upsampling with stride > 1.
            # Consider nn.ConvTranspose2d in TCN for upsampling if needed.
            print("Warning: temporal_strides for Decoder with GAT_TCN_Block might not perform T-upsampling as expected.")
            self.temporal_strides = temporal_strides
        else:
            self.temporal_strides = temporal_strides

        self.build_model()

    def build_model(self):
        input_channels = self.hidden_dimension # Input from bottleneck/latent space
        model_layers = nn.ModuleList()

        if not isinstance(self.adjacency_matrix, torch.Tensor):
            current_adj_matrix = torch.from_numpy(self.adjacency_matrix).float()
        else:
            current_adj_matrix = self.adjacency_matrix

        if current_adj_matrix.ndim == 3:
            # print(f"Decoder: Using the first adjacency matrix strategy. Shape: {current_adj_matrix.shape}")
            self.processed_adj_matrix = current_adj_matrix[0]
        elif current_adj_matrix.ndim == 2:
            self.processed_adj_matrix = current_adj_matrix
        else:
            raise ValueError(f"Adjacency matrix has unexpected dimensions: {current_adj_matrix.shape}")

        # layer_channels_for_gat_tcn = e.g. [hidden_dim_encoder_out, intermediate1, intermediate2, final_output_dim]
        # GAT_TCN_Block(input_channels, layer_channels_for_gat_tcn[0], ...)
        # GAT_TCN_Block(layer_channels_for_gat_tcn[0], layer_channels_for_gat_tcn[1], ...)

        current_block_input_channels = self.hidden_dimension
        for i, out_channels_block in enumerate(self.layer_channels_for_gat_tcn):
            model_layers.append(
                GAT_TCN_Block(
                    in_channels=current_block_input_channels,
                    out_channels=out_channels_block,
                    num_heads_gat=self.num_heads_gat,
                    temporal_kernel_size=9, # Can be configurable
                    temporal_stride=self.temporal_strides[i], # Typically 1 for decoder
                    dropout=self.dropout,
                    A_is_dense=True
                )
            )
            current_block_input_channels = out_channels_block
        self.model_layers = model_layers

    def forward(self, X: torch.Tensor, t: torch.Tensor = None) -> torch.Tensor:
        # X expected shape: (N_actual, hidden_dimension_encoder, T, V)
        device = X.device
        adj_matrix_on_device = self.processed_adj_matrix.to(device)
        for layer in self.model_layers:
            X = layer(X, adj_matrix_on_device)
        return X # Output shape: (N_actual, self.output_dim, T_final, V)


class DecoderResiduals(Decoder): # This class might need more significant changes for GAT
    def build_model(self) -> None:
        super().build_model()
        # This out layer might not be appropriate if GAT_TCN already outputs correct channels
        # And it operates on T dimension, which GAT_TCN doesn't change if stride=1
        # self.out = nn.Linear(self.n_frames, self.n_frames)
        # If the GAT_TCN_Block's last layer outputs self.output_dim, this fc might not be needed.
        # Or if it's a final pointwise linear projection on channels:
        # last_gat_tcn_out_channels = self.layer_channels_for_gat_tcn[-1]
        # self.final_projection = nn.Conv2d(last_gat_tcn_out_channels, self.output_dim, 1)
        print("DecoderResiduals: Review the 'out' layer and residual connections compatibility with GAT_TCN_Block.")


    def forward(self, X: torch.Tensor, t: torch.Tensor, residuals: List[torch.Tensor]) -> torch.Tensor:
        # X shape: (N*M, hidden_dim_encoder, T, V)
        # Residuals: List of [(N*M, C_res, T_res, V_res), ...] from Encoder
        device = X.device
        adj_matrix_on_device = self.processed_adj_matrix.to(device)

        if len(residuals) != len(self.model_layers):
            # This can happen if encoder/decoder layer counts differ or if residuals are structured differently
            # For skip connections, usually encoder and decoder have symmetric number of blocks/stages
            # For now, let's assume residuals align if counts are same.
            # print(f"Warning: Mismatch in number of residuals ({len(residuals)}) and decoder layers ({len(self.model_layers)}). Skipping residual connections.")
            pass # Fallback to simple decoder forward if residuals don't match

        for i, layer in enumerate(self.model_layers):
            X = layer(X, adj_matrix_on_device)
            if residuals: # Check if residuals list is not empty
                 # Ensure residuals are popped in correct order (reverse of encoder's storage)
                 # And that shapes are compatible for addition.
                 # GAT_TCN_Block preserves T,V dims if stride=1. Channels must match.
                res_to_add = residuals.pop() # Pops from the end, which is encoder's earlier layers if stored sequentially
                if X.shape == res_to_add.shape:
                    X = X + res_to_add
                else:
                    # print(f"Warning: Shape mismatch for residual connection. X: {X.shape}, Res: {res_to_add.shape}. Skipping this residual.")
                    # Potentially add a projection layer to res_to_add if channel dims differ
                    pass


        # The original self.out layer was:
        # X = self.out(X.permute(0, 1, 3, 2).contiguous()).permute(0, 1, 3, 2).contiguous()
        # This is a linear layer applied across the time dimension for each (batch, channel, joint).
        # It might not be necessary if the GAT_TCN_Blocks handle channel transformations appropriately.
        # If a final channel adjustment is needed, a 1x1 Conv2d is more common after GCNs.
        # Example: if X.shape[-3] (channels) is not self.output_dim:
        # X = self.final_projection(X) # if final_projection is defined in build_model

        return X


class Denoiser(nn.Module):
    # This Denoiser is a simple MLP and doesn't depend on graph structure,
    # so it remains unchanged.
    def __init__(self, input_size:int, hidden_sizes:List[int], cond_size:int=None, bias:bool=True, device:Union[str, torch.DeviceObjType]='cpu') -> None:
        super().__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.cond_size = cond_size
        self.embedding_dim = cond_size if cond_size is not None else 128 # Default if no cond_size
        self.bias = bias
        self.device = device
        self.build_model()
        
    def build_model(self) -> None:
        self.net = nn.ModuleList()
        self.cond_layers = nn.ModuleList() if self.cond_size is not None else None # conditioning for t
        self.ext_cond_layers = nn.ModuleList() if self.cond_size is not None else None # conditioning for external cond

        n_layers = len(self.hidden_sizes)
        current_size = self.input_size
        for idx, next_dim in enumerate(self.hidden_sizes):
            layer = nn.Linear(current_size, next_dim, bias=self.bias)
            if idx == n_layers-1: # Last layer
                self.net.append(layer)
            else:
                self.net.append(nn.Sequential(layer,
                                              nn.BatchNorm1d(next_dim), 
                                              nn.ReLU(inplace=True)))
            if self.cond_size is not None:
                 # For time embedding 't'
                self.cond_layers.append(nn.Linear(self.embedding_dim, next_dim, bias=self.bias))
                 # For external condition 'cond' (e.g. from STSAE)
                self.ext_cond_layers.append(nn.Linear(self.cond_size, next_dim, bias=self.bias))

            current_size = next_dim
            
    def pos_encoding(self, t:torch.Tensor, channels:int) -> torch.Tensor:
        # t shape: (N, 1)
        # channels: self.embedding_dim
        inv_freq = 1.0 / (
            10000 ** (torch.arange(0, channels, 2, device=self.device).float() / channels)
        ) # shape: (channels/2)
        pos_enc_a = torch.sin(t * inv_freq) # (N, 1) * (channels/2) -> (N, channels/2)
        pos_enc_b = torch.cos(t * inv_freq) # (N, 1) * (channels/2) -> (N, channels/2)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1) # (N, channels)
        return pos_enc

    def forward(self, X:torch.Tensor, t:torch.Tensor, cond:torch.Tensor=None) -> torch.Tensor:
        # X: (N_actual, flattened_features) e.g. (N*M, latent_dim_from_stsae)
        # t: (N_actual, 1) or (N_actual) for time steps
        # cond: (N_actual, cond_size) external condition (e.g. Z from STSAE)
        
        if t.ndim == 1:
            t = t.unsqueeze(-1) # Ensure t is (N_actual, 1)
        t_emb = self.pos_encoding(t, self.embedding_dim) # (N_actual, embedding_dim)

        for i in range(len(self.net)):
            X = self.net[i](X)
            if self.cond_layers is not None: # Add time embedding
                time_effect = self.cond_layers[i](t_emb)
                X = X + time_effect
            if cond is not None and self.ext_cond_layers is not None: # Add external condition
                ext_cond_effect = self.ext_cond_layers[i](cond)
                X = X + ext_cond_effect
        return X

Overwriting /kaggle/working/DCMD-main/models/common/components.py


In [14]:
%%writefile /kaggle/working/DCMD-main/models/gcae/graph.py
import numpy as np

class Graph():
    """ The Graph to model the skeletons of human body/hand

    Args:
        layout (str): must be one of 'ntu-rgb+d', 'h36m', 'coco', 'hands'
            The layout of skeletons. ('ntu-rgb+d' by default)
        strategy (str): must be one of 'uniform', 'distance', 'spatial', 'binary'
            The strategy to generate adjacency matrix. ('uniform' by default)
        max_hop (int): the maximal hop graph to consider. (1 by default)
        dilation (int): the dilation of convolution operator. (1 by default)
        normalize (bool): whether to normalize the adjacency matrix. (True by default)
    """

    def __init__(self,
                 layout='ntu-rgb+d',
                 strategy='uniform',
                 max_hop=1,
                 dilation=1,
                 normalize=True):
        self.max_hop = max_hop
        self.dilation = dilation
        self.normalize = normalize
        self.num_node, self.edge, self.center = self._get_layout_info(layout)
        self.hop_dis = self._get_hop_distance(self.num_node, self.edge, max_hop=max_hop)
        self.A = self._get_adjacency(strategy)

    def __str__(self):
        return self.A

    def _get_layout_info(self, layout):
        if layout == 'ntu-rgb+d':
            num_node = 25
            self_link = [(i, i) for i in range(num_node)]
            inward_orange_idx = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                               (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                               (11, 10), (12, 11), (13, 1), (14, 13),
                               (15, 14), (16, 15), (17, 1), (18, 17),
                               (19, 18), (20, 19), (22, 23), (23, 8),
                               (24, 25), (25, 12)]
            inward = [(i - 1, j - 1) for (i, j) in inward_orange_idx]
            outward = [(j, i) for (i, j) in inward]
            edge = self_link + inward + outward
            center = 21 - 1
        elif layout == 'h36m':
            num_node = 17
            self_link = [(i, i) for i in range(num_node)]
            # Spine, Neck, Head, HeadTopt
            # Left: Shoulder, Arm, ForeArm, Hand
            # Right: Shoulder, Arm, ForeArm, Hand
            # Hip, Left: Thigh, Shin, Foot
            # Hip, Right: Thigh, Shin, Foot
            edge_human36m = [(0, 1), (1, 2), (2, 3), (0, 4), (4, 5), (5, 6), (0, 7), (7, 8),
                        (8, 9), (9, 10), (8, 11), (11, 12), (12, 13), (8, 14),
                        (14, 15), (15, 16)]
            edge = self_link + edge_human36m + [(j,i) for (i,j) in edge_human36m]
            center = 0 # Hip
        elif layout == 'coco':
            num_node = 17 # Original COCO has 17 keypoints
            self_link = [(i, i) for i in range(num_node)]
            # Nose, Leye, Reye, Lear, Rear, Lshoulder, Rshoulder, Lelbow, Relbow, Lwrist, Rwrist, Lhip, Rhip, Lknee, Rknee, Lankle, Rankle
            coco_edges = [
                (0, 1), (0, 2), (1, 3), (2, 4),  # Head
                (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # Arms
                (11, 12), (5, 11), (6, 12),  # Torso
                (11, 13), (13, 15), (12, 14), (14, 16)  # Legs
            ]
            edge = self_link + coco_edges + [(j,i) for (i,j) in coco_edges]
            center = 0 # Nose, or an implicit center derived from hips/shoulders
        elif layout == 'hands':
            num_node = 21 # For one hand usually
            self_link = [(i, i) for i in range(num_node)]
            # Wrist, Thumb1, Thumb2, Thumb3, Thumb4, Index1,...Pinky4
            hand_edges = [
                (0,1), (1,2), (2,3), (3,4), #Thumb
                (0,5), (5,6), (6,7), (7,8), #Index
                (0,9), (9,10), (10,11), (11,12), #Middle
                (0,13), (13,14), (14,15), (15,16), #Ring
                (0,17), (17,18), (18,19), (19,20) #Pinky
            ]
            edge = self_link + hand_edges + [(j,i) for (i,j) in hand_edges]
            center = 0 # Wrist
        else:
            raise ValueError(f"Do Not Exist This Layout: {layout}")
        return num_node, edge, center

    def _get_hop_distance(self, num_node, edge, max_hop=1):
        A = np.zeros((num_node, num_node))
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1

        # compute hop steps
        hop_dis = np.zeros((num_node, num_node)) + np.inf
        transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
        arrive_mat = (np.stack(transfer_mat) > 0)
        for d in range(max_hop, -1, -1):
            hop_dis[arrive_mat[d]] = d
        return hop_dis

    def _get_adjacency(self, strategy):
        valid_hop = range(0, self.max_hop + 1, self.dilation)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = np.zeros((self.num_node, self.num_node))
        if strategy == 'uniform':
            for hop in valid_hop:
                normalize_adjacency[self.hop_dis == hop] = 1
            if self.normalize:
                D = np.sum(normalize_adjacency, axis=0)
                normalize_adjacency = normalize_adjacency / (D + 1e-6) # Add epsilon to avoid division by zero
            A = normalize_adjacency
        elif strategy == 'distance':
            for hop in valid_hop:
                normalize_adjacency[self.hop_dis == hop] = 1.0 / (hop + 1) # Closer hops have higher weight
            if self.normalize:
                D = np.sum(normalize_adjacency, axis=0)
                normalize_adjacency = normalize_adjacency / (D + 1e-6)
            A = normalize_adjacency
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = 1
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = 1
                            else:
                                a_further[j, i] = 1
                if self.normalize:
                    for a in [a_root, a_close, a_further]:
                        D = np.sum(a, axis=0)
                        a = a / (D + 1e-6)
                A.append(a_root)
                A.append(a_close)
                A.append(a_further)
            A = np.stack(A) # Results in (num_strategies, num_node, num_node)
        elif strategy == 'binary': # Adjacency matrix with 0 or 1
            A = np.zeros((self.num_node, self.num_node))
            for i,j in self.edge: # only direct connections
                if i !=j: # exclude self-loops from basic binary unless explicitly added
                    A[i,j] = 1
                    A[j,i] = 1
            # if you want self-loops in binary:
            # for i in range(self.num_node):
            #   A[i,i] = 1
            # No normalization for binary usually, but can be added if needed
        else:
            raise ValueError("Do Not Exist This Strategy")

        # If A is a list of matrices (e.g. for 'spatial' strategy),
        # and the model expects a single matrix, you might need to select one or combine them.
        # For GAT_TCN_Block expecting A_is_dense=True and a single Adjacency:
        if isinstance(A, np.ndarray) and A.ndim == 3:
            # print(f"Graph strategy '{strategy}' produced multiple adjacency matrices ({A.shape[0]}). Using the first one.")
            A = A[0] # Take the first one (e.g., root connections for 'spatial') or sum/mean them.
                     # The original ST-GCN could handle multiple A's by creating parallel GCN layers.
                     # Our GAT_TCN_Block currently uses one A.
        return A

Writing /kaggle/working/DCMD-main/models/gcae/graph.py
